### Importing relevant Libraries

In [2]:
import pandas as pd
import io
import os
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import kendalltau
import scipy.stats

from scipy.stats import pearsonr, spearmanr, kendalltau
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.signal import hilbert
from scipy.stats import skew, kurtosis
import mplfinance as mpf
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel
from sklearn.model_selection import train_test_split
import plotly.graph_objs as go
import kaleido
import talib
import warnings
warnings.simplefilter("ignore")




## Data Preparation

In [3]:
df_3m = pd.read_csv('btcusdt_3m_train.csv')
df_3m.rename(columns={'datetime': 'Date'}, inplace=True)
df_3m['Date'] = pd.to_datetime(df_3m['Date'])
df_3m = df_3m.drop('Unnamed: 0', axis=1) 
df_3m.set_index('Date', inplace=True)
df_3m['return'] = df_3m['close'].shift(-1) / df_3m['close'] - 1
df_3m['return_abs'] = abs(df_3m['close'].shift(-1) / df_3m['close'] - 1)
df_3m['return_2'] = df_3m['close'].shift(-2) / df_3m['close'] - 1
df_3m['return_2_abs'] = abs(df_3m['close'].shift(-2) / df_3m['close'] - 1)


In [4]:
df_5m = pd.read_csv('btcusdt_5m_train.csv')
df_5m.rename(columns={'datetime': 'Date'}, inplace=True)
df_5m['Date'] = pd.to_datetime(df_5m['Date'])
df_5m = df_5m.drop('Unnamed: 0', axis=1) 
df_5m.set_index('Date', inplace=True)
df_5m['return'] = df_5m['close'].shift(-1) / df_5m['close'] - 1
df_5m['return_abs'] = abs(df_5m['close'].shift(-1) / df_5m['close'] - 1)
df_5m['return_2'] = df_5m['close'].shift(-2) / df_5m['close'] - 1
df_5m['return_2_abs'] = abs(df_5m['close'].shift(-2) / df_5m['close'] - 1)

In [5]:
df_15m = pd.read_csv('btcusdt_15m_train.csv')
df_15m.rename(columns={'datetime': 'Date'}, inplace=True)
df_15m['Date'] = pd.to_datetime(df_15m['Date'])
df_15m = df_15m.drop('Unnamed: 0', axis=1) 
df_15m.set_index('Date', inplace=True)
df_15m['return'] = df_15m['close'].shift(-1) / df_15m['close'] - 1
df_15m['return_abs'] = abs(df_15m['close'].shift(-1) / df_15m['close'] - 1)
df_15m['return_2'] = (df_15m['close'].shift(-2) / df_15m['close'] - 1)
df_15m['return_2_abs'] = abs(df_15m['close'].shift(-2) / df_15m['close'] - 1)



In [6]:
df_30m = pd.read_csv('btcusdt_30m_train.csv')
df_30m.rename(columns={'datetime': 'Date'}, inplace=True)
df_30m['Date'] = pd.to_datetime(df_30m['Date'])
df_30m = df_30m.drop('Unnamed: 0', axis=1) 
df_30m.set_index('Date', inplace=True)
df_30m['return'] = df_30m['close'].shift(-1) / df_30m['close'] - 1
df_30m['return_2'] = df_30m['close'].shift(-2) / df_30m['close'] - 1
df_30m['return_abs'] = abs(df_30m['close'].shift(-1) / df_30m['close'] - 1)
df_30m['return_2_abs'] = abs(df_30m['close'].shift(-2) / df_30m['close'] - 1)

In [7]:
df_1h = pd.read_csv('btcusdt_1h_train.csv')
df_1h.rename(columns={'datetime': 'Date'}, inplace=True)
df_1h['Date'] = pd.to_datetime(df_1h['Date'])
df_1h = df_1h.drop('Unnamed: 0', axis=1) 
df_1h.set_index('Date', inplace=True)
df_1h['return'] = df_1h['close'].shift(-1) / df_1h['close'] - 1
df_1h['return_2'] = df_1h['close'].shift(-2) / df_1h['close'] - 1
df_1h['return_abs'] = abs(df_1h['close'].shift(-1) / df_1h['close'] - 1)
df_1h['return_abs_2'] = abs(df_1h['close'].shift(-2) / df_1h['close'] - 1)


In [8]:
daily_df = df_1h.resample('D').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
daily_df['return'] = daily_df['close'].shift(-1) / daily_df['close'] - 1

In [9]:
def calculate_correlation(df):
    # Check if 'Signal' or 'Return' columns have constant values
    if df['Signal'].nunique() <= 1 or df['return'].nunique() <= 1:
        return "Insufficient variation in 'Signal' or 'Return' for correlation calculation."

    # Drop NaN values for correlation calculation
    df = df.dropna(subset=['return'])

    # Calculate Spearman and Kendall correlations
    spearman_corr, spearman_pval = spearmanr(df['Signal'], df['return'])
    kendall_corr, kendall_pval = kendalltau(df['Signal'], df['return'])

    # Format the output as percentages
    formatted_results = {
        'Spearman Correlation': f"{spearman_corr*100:.4f}%",
        'Spearman P-value': f"{spearman_pval*100}%",
        'Kendall Correlation': f"{kendall_corr*100:.4f}%",
        'Kendall P-value': f"{kendall_pval*100}%"
    }
    return formatted_results

## Momentum Oscillators:

### List of major indicators
1. Relative Strength Index (RSI): RSI measures overbought or oversold conditions, aiding traders in timing entries and exits. 

2. Stochastic Oscillator: Stochastic Oscillator identifies potential reversal points, helping traders spot trend changes.

3. Rate of Change (ROC): ROC calculates price change percentage, assisting traders in assessing momentum. 

4. Williams %R: Williams %R measures overbought or oversold levels, aiding traders in timing trades.

5. Commodity Channel Index (CCI): CCI identifies trend reversals, helping traders spot potential turning points.

6. Money Flow Index (MFI): MFI evaluates buying and selling pressure, aiding traders in assessing market sentiment.

7. Ultimate Oscillator: The Ultimate Oscillator combines multiple timeframes to provide a comprehensive momentum analysis.

8. True Strength Index (TSI): TSI measures the price rate of change, offering insights into trend strength.

9. Detrended Price Oscillator (DPO): DPO removes trend from prices, helping traders identify price cycles.

10. Percentage Price Oscillator (PPO): PPO compares moving averages, assisting traders in analyzing trend momentum.

11. Price Rate of Change (PROC): PROC calculates the price change rate, helping traders gauge momentum.

12. Relative Vigor Index (RVI): RVI analyzes trend direction, assisting traders in assessing market strength.

13. Klinger Oscillator: The Klinger Oscillator combines volume and price data to provide momentum insights.

14. Chande's Momentum Oscillator: Chande's Momentum Oscillator helps traders identify trend strength and potential reversals.

15. Schaff Trend Cycle: Schaff Trend Cycle smooths and signals trends, aiding traders in trend analysis.

#### These 15 indicators have been coded without use of any library have used standard metrics to generate a signal line and/or log column

### Stochastic Oscillator

In [10]:
def calculate_stochastic_oscillator(df, k_periods=14, d_periods=3):
    """
    Calculate the Stochastic Oscillator for given OHLCV data and provide trading signals.

    :param df: DataFrame with columns ['open', 'high', 'low', 'close', 'volume']
    :param k_periods: The number of periods to include in %K calculation
    :param d_periods: The number of periods for the %D moving average
    :return: DataFrame with additional columns ['%K', '%D', 'Signal']
    """
    # Calculate %K
    low_min = df['low'].rolling(window=k_periods).min()
    high_max = df['high'].rolling(window=k_periods).max()
    df['%K'] = ((df['close'] - low_min) / (high_max - low_min)) * 100

    # Calculate %D as moving average of %K
    df['%D'] = df['%K'].rolling(window=d_periods).mean()

    # Generate trading signals
    df['Signal'] = int(0)
    df.loc[(df['%K'] > df['%D']) & (df['%K'].shift(1) <= df['%D'].shift(1)), 'Signal'] = int(1)
    df.loc[(df['%K'] < df['%D']) & (df['%K'].shift(1) >= df['%D'].shift(1)), 'Signal'] = int(-1)

    return df

In [11]:
# Calculate Stochastic Oscillator
stochastic_df = calculate_stochastic_oscillator(df_1h)
calculate_correlation(stochastic_df)



{'Spearman Correlation': '-2.6890%',
 'Spearman P-value': '0.0003430607486071195%',
 'Kendall Correlation': '-2.0910%',
 'Kendall P-value': '0.00035904622809980843%'}

Interpretation
Value: -2.6890%
Interpretation: This indicates a very slight negative correlation between the 'Signal' and 'Return' columns.
P-value: 0.0003%
Statistical Significance: This extremely low p-value indicates that the correlation result is statistically significant, meaning it is unlikely to have occurred by chance.


### RSI

In [12]:
def calculate_relative_strength_index(df, periods=14):
    """
    Calculate the Relative Strength Index (RSI) for given OHLCV data.

    :param df: DataFrame with columns ['open', 'high', 'low', 'close', 'volume']
    :param periods: The number of periods to include in RSI calculation
    :return: DataFrame with an additional column ['RSI']
    """
    delta = df['close'].diff()

    # Make two series: one for gains and one for losses
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()

    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Generate trading signals based on RSI
    # Overbought signal (sell): RSI >= 70
    # Oversold signal (buy): RSI <= 30
    df['Signal'] = int(0)
    df.loc[df['RSI'] >= 70, 'Signal'] = int(-1)
    df.loc[df['RSI'] <= 30, 'Signal'] = int(1)

    return df

In [13]:
df_rsi=calculate_relative_strength_index(df_1h)

In [14]:
calculate_correlation(df_rsi)

{'Spearman Correlation': '1.2405%',
 'Spearman P-value': '3.222474214871845%',
 'Kendall Correlation': '1.0074%',
 'Kendall P-value': '2.846931292686584%'}

### Rate of Change (ROC)

In [15]:
def calculate_rate_of_change(df, n=12):
    """
    Calculate the Rate of Change (ROC) indicator for given OHLCV data.

    :param df: DataFrame with columns ['open', 'high', 'low', 'close', 'volume']
    :param n: The number of periods to use in the ROC calculation
    :return: DataFrame with an additional column ['ROC']
    """
    df['ROC'] = ((df['close'] - df['close'].shift(n)) / df['close'].shift(n)) * 100

    # Define overbought and oversold levels based on historical data. 
    # These levels should be adjusted based on specific asset characteristics and historical performance.
    overbought_level = 10  # Example value
    oversold_level = -10   # Example value

    # Generate trading signals
    df['Signal'] = 0
    df.loc[(df['ROC'] < oversold_level) & (df['ROC'].shift(1) >= oversold_level), 'Signal'] = 1 # Go long
    df.loc[(df['ROC'] > overbought_level) & (df['ROC'].shift(1) <= overbought_level), 'Signal'] = -1 # Go short

    return df
# Calculate Rate of Change for the sample data
roc_df = calculate_rate_of_change(df_1h)

# Assuming the 'return' column is already in the DataFrame, you can now use the calculate_correlation function
correlation_results = calculate_correlation(roc_df)
print(correlation_results)

{'Spearman Correlation': '2.3451%', 'Spearman P-value': '0.0051396818836907604%', 'Kendall Correlation': '1.9154%', 'Kendall P-value': '0.005075665515106395%'}


### Williams %R


In [16]:
def calculate_williams_r(df, n=14):
    """
    Calculate the Williams %R for given OHLCV data.

    :param df: DataFrame with columns ['open', 'high', 'low', 'close', 'volume']
    :param n: The number of periods to use in the Williams %R calculation
    :return: DataFrame with an additional column ['Williams %R']
    """
    highest_high = df['high'].rolling(n).max()
    lowest_low = df['low'].rolling(n).min()
    df['Williams %R'] = -100 * ((highest_high - df['close']) / (highest_high - lowest_low))

    # Generate trading signals
    # Overbought signal (sell): Williams %R above -20
    # Oversold signal (buy): Williams %R below -80
    df['Signal'] = 0
    df.loc[df['Williams %R'] > -20, 'Signal'] = -1  # Sell
    df.loc[df['Williams %R'] < -80, 'Signal'] = 1   # Buy

    return df
williams_r_df = calculate_williams_r(df_1h)
correlation_results = calculate_correlation(williams_r_df)
print(correlation_results)

{'Spearman Correlation': '6.7257%', 'Spearman P-value': '3.098401439448174e-29%', 'Kendall Correlation': '5.4354%', 'Kendall P-value': '1.4063887170831725e-30%'}


### Commodity Channel Index (CCI)


In [17]:
def calculate_commodity_channel_index(df, n=20):
    """
    Calculate the Commodity Channel Index (CCI) for given OHLCV data.

    :param df: DataFrame with columns ['high', 'low', 'close']
    :param n: The number of periods to use in the CCI calculation
    :return: DataFrame with an additional column ['CCI']
    """
    TP = (df['high'] + df['low'] + df['close']) / 3
    TPMA = TP.rolling(n).mean()
    MD = TP.rolling(n).apply(lambda x: np.fabs(x - x.mean()).mean())
    df['CCI'] = (TP - TPMA) / (MD * 0.015)

    # Generate trading signals based on CCI
    overbought = 100
    oversold = -100
    df['Signal'] = 0
    df.loc[df['CCI'] > overbought, 'Signal'] = -1  # Go short
    df.loc[df['CCI'] < oversold, 'Signal'] = 1    # Go long

    return df


# Calculate Commodity Channel Index for the sample data
cci_df = calculate_commodity_channel_index(df_1h)

correlation_results = calculate_correlation(cci_df)
print(correlation_results)



{'Spearman Correlation': '3.9605%', 'Spearman P-value': '7.91343347716755e-10%', 'Kendall Correlation': '3.1632%', 'Kendall P-value': '2.9903900125621704e-10%'}


### Money Flow Index (MFI)

In [18]:
def calculate_money_flow_index(df, n=14):
    """
    Calculate the Money Flow Index (MFI) for given OHLCV data.

    :param df: DataFrame with columns ['high', 'low', 'close', 'volume']
    :param n: The number of periods to use in the MFI calculation
    :return: DataFrame with an additional column ['MFI']
    """
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    raw_money_flow = typical_price * df['volume']
    money_flow_ratio = (
        raw_money_flow.rolling(n).apply(lambda x: x[x > x.shift(1)].sum()) /
        raw_money_flow.rolling(n).apply(lambda x: x[x < x.shift(1)].sum())
    )
    df['MFI'] = 100 - 100 / (1 + money_flow_ratio)

    # Generate trading signals based on MFI
    overbought = 80
    oversold = 20
    df['Signal'] = 0
    df.loc[df['MFI'] > overbought, 'Signal'] = -1  # Go short
    df.loc[df['MFI'] < oversold, 'Signal'] = 1    # Go long

    return df

mfi_df = calculate_money_flow_index(df_1h)

correlation_results = calculate_correlation(mfi_df)
print(correlation_results)

{'Spearman Correlation': '0.0129%', 'Spearman P-value': '98.2224315774746%', 'Kendall Correlation': '0.0105%', 'Kendall P-value': '98.22978431583147%'}


### Ultimate Oscillator

In [19]:
def calculate_ultimate_oscillator(df, short_period=7, medium_period=14, long_period=28):
    """
    Calculate the Ultimate Oscillator for given OHLCV data.
    The Ultimate Oscillator combines multiple timeframes to provide a comprehensive momentum analysis.

    :param df: DataFrame with columns ['high', 'low', 'close']
    :param short_period: The short period for the Ultimate Oscillator calculation
    :param medium_period: The medium period for the Ultimate Oscillator calculation
    :param long_period: The long period for the Ultimate Oscillator calculation
    :return: DataFrame with an additional column ['Ultimate Oscillator']
    """
    # Buying Pressure (BP) and True Range (TR)
    BP = df['close'] - df['low'].rolling(min_periods=1, window=2).min()
    TR = df['high'].rolling(min_periods=1, window=2).max() - df['low'].rolling(min_periods=1, window=2).min()

    # Average of BP and TR for different periods
    avg_BP_short = BP.rolling(window=short_period).sum()
    avg_TR_short = TR.rolling(window=short_period).sum()
    avg_BP_medium = BP.rolling(window=medium_period).sum()
    avg_TR_medium = TR.rolling(window=medium_period).sum()
    avg_BP_long = BP.rolling(window=long_period).sum()
    avg_TR_long = TR.rolling(window=long_period).sum()

    # Ultimate Oscillator
    df['Ultimate Oscillator'] = (
        (4 * avg_BP_short / avg_TR_short) +
        (2 * avg_BP_medium / avg_TR_medium) +
        (avg_BP_long / avg_TR_long)
    ) / 7

    # Generate trading signals based on Ultimate Oscillator
    overbought = 70
    oversold = 30
    df['Signal'] = 0
    df.loc[df['Ultimate Oscillator'] > overbought, 'Signal'] = -1  # Go short
    df.loc[df['Ultimate Oscillator'] < oversold, 'Signal'] = 1    # Go long

    return df

# Calculate Ultimate Oscillator for the sample data
uo_df = calculate_ultimate_oscillator(df_1h)

correlation_results = calculate_correlation(uo_df)
print(correlation_results)

uo_df



{'Spearman Correlation': '0.1548%', 'Spearman P-value': '78.93277038921023%', 'Kendall Correlation': '0.1264%', 'Kendall P-value': '78.93226507540271%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,%D,Signal,RSI,ROC,Williams %R,CCI,MFI,Ultimate Oscillator
Date,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,81.697443,1,66.572202,5.473474,-21.049936,84.833974,42.429801,0.585379
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,83.662325,1,78.548567,7.166195,-10.142469,104.420576,35.973714,0.580672
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,86.173262,1,81.140590,7.691147,-10.287809,114.564170,44.285557,0.604634


### True Strength Index (TSI)

In [20]:
def calculate_true_strength_index(df, high_period=25, low_period=13):
    """
    Calculate the True Strength Index (TSI) for given OHLCV data.
    TSI is a momentum oscillator that measures the rate of change of price.

    :param df: DataFrame with column ['close']
    :param high_period: The high period for the TSI calculation
    :param low_period: The low period for the TSI calculation
    :return: DataFrame with an additional column ['TSI']
    """
    delta_close = df['close'].diff()

    # Double smoothed price change
    smoothed_pc = delta_close.rolling(window=high_period).mean()
    double_smoothed_pc = smoothed_pc.rolling(window=low_period).mean()

    # Double smoothed absolute price change
    abs_smoothed_pc = abs(delta_close).rolling(window=high_period).mean()
    double_smoothed_abs_pc = abs_smoothed_pc.rolling(window=low_period).mean()

    # True Strength Index
    df['TSI'] = 100 * (double_smoothed_pc / double_smoothed_abs_pc)

    # Generate trading signals based on TSI
    overbought = 25   # Example value, to be adjusted based on specific analysis
    oversold = -25    # Example value, to be adjusted based on specific analysis
    df['Signal'] = 0
    df.loc[df['TSI'] > overbought, 'Signal'] = -1  # Go short
    df.loc[df['TSI'] < oversold, 'Signal'] = 1    # Go long

    return df

# Calculate True Strength Index for the sample data
tsi_df = calculate_true_strength_index(df_1h)

correlation_results = calculate_correlation(tsi_df)
print(correlation_results)

tsi_df



{'Spearman Correlation': '-1.2080%', 'Spearman P-value': '3.701396993981217%', 'Kendall Correlation': '-0.9663%', 'Kendall P-value': '3.4267555194373855%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,%D,Signal,RSI,ROC,Williams %R,CCI,MFI,Ultimate Oscillator,TSI
Date,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,81.697443,0,66.572202,5.473474,-21.049936,84.833974,42.429801,0.585379,5.932767
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,83.662325,0,78.548567,7.166195,-10.142469,104.420576,35.973714,0.580672,6.942538
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,86.173262,0,81.140590,7.691147,-10.287809,114.564170,44.285557,0.604634,8.998241


### Detrended Price Oscillator (DPO)

In [21]:
def calculate_detrended_price_oscillator(df, n=20):
    """
    Calculate the Detrended Price Oscillator (DPO) for given OHLCV data.
    DPO removes trend from prices, helping traders identify price cycles.

    :param df: DataFrame with column ['close']
    :param n: The number of periods to use in the DPO calculation
    :return: DataFrame with an additional column ['DPO']
    """
    displaced_ma = df['close'].rolling(window=n).mean().shift(-n//2 + 1)
    df['DPO'] = df['close'] - displaced_ma

    # Generate trading signals based on DPO
    # Note: Trading signals for DPO are typically based on the oscillator crossing above or below zero, or divergences
    df['Signal'] = 0
    df.loc[df['DPO'] > 0, 'Signal'] = 1  # Potential buy signal
    df.loc[df['DPO'] < 0, 'Signal'] = -1 # Potential sell signal

    return df

# Calculate Detrended Price Oscillator for the sample data
dpo_df = calculate_detrended_price_oscillator(df_1h)

correlation_results = calculate_correlation(dpo_df)
print(correlation_results)

dpo_df




{'Spearman Correlation': '-28.2774%', 'Spearman P-value': '0.0%', 'Kendall Correlation': '-23.0774%', 'Kendall P-value': '0.0%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,%D,Signal,RSI,ROC,Williams %R,CCI,MFI,Ultimate Oscillator,TSI,DPO
Date,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,81.697443,0,66.572202,5.473474,-21.049936,84.833974,42.429801,0.585379,5.932767,NaN
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,83.662325,0,78.548567,7.166195,-10.142469,104.420576,35.973714,0.580672,6.942538,NaN
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,86.173262,0,81.140590,7.691147,-10.287809,114.564170,44.285557,0.604634,8.998241,NaN



### Percentage Price Oscillator (PPO)


In [22]:
def calculate_percentage_price_oscillator(df, fast_period=12, slow_period=26, signal_period=9):
    """
    Calculate the Percentage Price Oscillator (PPO) for given OHLCV data.
    PPO compares moving averages, assisting traders in analyzing trend momentum.

    :param df: DataFrame with column ['close']
    :param fast_period: The fast EMA period for PPO calculation
    :param slow_period: The slow EMA period for PPO calculation
    :param signal_period: The signal line EMA period
    :return: DataFrame with additional columns ['PPO', 'Signal Line']
    """
    # Calculate fast and slow EMAs
    ema_fast = df['close'].ewm(span=fast_period, adjust=False).mean()
    ema_slow = df['close'].ewm(span=slow_period, adjust=False).mean()

    # Calculate PPO and its signal line
    df['PPO'] = ((ema_fast - ema_slow) / ema_slow) * 100
    df['Signal Line'] = df['PPO'].ewm(span=signal_period, adjust=False).mean()

    # Generate trading signals based on PPO
    df['Signal'] = 0
    df.loc[df['PPO'] > df['Signal Line'], 'Signal'] = 1  # Potential buy signal
    df.loc[df['PPO'] < df['Signal Line'], 'Signal'] = -1 # Potential sell signal

    return df

# Calculate Percentage Price Oscillator for the sample data
ppo_df = calculate_percentage_price_oscillator(df_1h)
correlation_results = calculate_correlation(ppo_df)
print(correlation_results)




ppo_df



{'Spearman Correlation': '-2.3856%', 'Spearman P-value': '0.003802753273045419%', 'Kendall Correlation': '-1.9478%', 'Kendall P-value': '0.0038141006923877304%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,RSI,ROC,Williams %R,CCI,MFI,Ultimate Oscillator,TSI,DPO,PPO,Signal Line
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.192536,-0.038507
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.266355,-0.084077
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.273922,-0.122046
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.163563,-0.130349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,66.572202,5.473474,-21.049936,84.833974,42.429801,0.585379,5.932767,NaN,0.955580,0.494687
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,78.548567,7.166195,-10.142469,104.420576,35.973714,0.580672,6.942538,NaN,1.059104,0.607570
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,81.140590,7.691147,-10.287809,114.564170,44.285557,0.604634,8.998241,NaN,1.161675,0.718391



### Price Rate of Change (PROC)


In [23]:
def calculate_price_rate_of_change(df, n=14):
    """
    Calculate the Price Rate of Change (PROC) for given OHLCV data.
    PROC calculates the price change rate, helping traders gauge momentum.

    :param df: DataFrame with column ['close']
    :param n: The number of periods to use in the PROC calculation
    :return: DataFrame with an additional column ['PROC']
    """
    df['PROC'] = df['close'].pct_change(periods=n) * 100

    # Generate trading signals based on PROC
    # Note: Trading signals for PROC are typically based on the oscillator crossing above or below a certain threshold
    df['Signal'] = 0
    df.loc[df['PROC'] > 0, 'Signal'] = 1  # Potential buy signal
    df.loc[df['PROC'] < 0, 'Signal'] = -1 # Potential sell signal

    return df

# Calculate Price Rate of Change for the sample data
proc_df = calculate_price_rate_of_change(df_1h,25)
correlation_results = calculate_correlation(proc_df)
print(correlation_results)

proc_df



{'Spearman Correlation': '-2.8827%', 'Spearman P-value': '6.436314791159865e-05%', 'Kendall Correlation': '-2.3513%', 'Kendall P-value': '6.563444838957684e-05%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,ROC,Williams %R,CCI,MFI,Ultimate Oscillator,TSI,DPO,PPO,Signal Line,PROC
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.192536,-0.038507,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.266355,-0.084077,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.273922,-0.122046,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.163563,-0.130349,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,5.473474,-21.049936,84.833974,42.429801,0.585379,5.932767,NaN,0.955580,0.494687,2.644655
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,7.166195,-10.142469,104.420576,35.973714,0.580672,6.942538,NaN,1.059104,0.607570,4.563801
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,7.691147,-10.287809,114.564170,44.285557,0.604634,8.998241,NaN,1.161675,0.718391,4.623390



### Relative Vigor Index (RVI)



In [24]:
def calculate_relative_vigor_index(df, n=10):
    """
    Calculate the Relative Vigor Index (RVI) for given OHLCV data.
    RVI analyzes trend direction, assisting traders in assessing market strength.

    :param df: DataFrame with columns ['open', 'high', 'low', 'close']
    :param n: The number of periods to use in the RVI calculation
    :return: DataFrame with additional columns ['RVI', 'RVI Signal']
    """
    # RVI calculation
    numerator = ((df['close'] - df['open']) + 2 * (df['close'].shift(1) - df['open'].shift(1)) +
                 2 * (df['close'].shift(2) - df['open'].shift(2)) + (df['close'].shift(3) - df['open'].shift(3))) / 6
    denominator = ((df['high'] - df['low']) + 2 * (df['high'].shift(1) - df['low'].shift(1)) +
                   2 * (df['high'].shift(2) - df['low'].shift(2)) + (df['high'].shift(3) - df['low'].shift(3))) / 6

    df['RVI'] = (numerator / denominator).rolling(window=n).mean()

    # RVI Signal line
    df['RVI Signal'] = df['RVI'].rolling(window=4).mean()

    # Generate trading signals based on RVI
    df['Signal'] = 0
    df.loc[df['RVI'] > df['RVI Signal'], 'Signal'] = 1  # Potential buy signal
    df.loc[df['RVI'] < df['RVI Signal'], 'Signal'] = -1 # Potential sell signal

    return df



# Calculate Relative Vigor Index for the sample data
rvi_df = calculate_relative_vigor_index(df_1h)
correlation_results = calculate_correlation(rvi_df)
print(correlation_results)

rvi_df



{'Spearman Correlation': '-4.6963%', 'Spearman P-value': '4.975625160013542e-14%', 'Kendall Correlation': '-3.8322%', 'Kendall P-value': '5.2789631544755325e-14%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,CCI,MFI,Ultimate Oscillator,TSI,DPO,PPO,Signal Line,PROC,RVI,RVI Signal
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.192536,-0.038507,NaN,NaN,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.266355,-0.084077,NaN,NaN,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.273922,-0.122046,NaN,NaN,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.163563,-0.130349,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,84.833974,42.429801,0.585379,5.932767,NaN,0.955580,0.494687,2.644655,0.288866,0.217017
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,104.420576,35.973714,0.580672,6.942538,NaN,1.059104,0.607570,4.563801,0.308502,0.265299
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,114.564170,44.285557,0.604634,8.998241,NaN,1.161675,0.718391,4.623390,0.317375,0.294668


### Klinger Oscillator



In [25]:
def calculate_klinger_oscillator(df, short_period=34, long_period=55):
    """
    Calculate the Klinger Oscillator for given OHLCV data.
    The Klinger Oscillator combines volume and price data to provide momentum insights.

    :param df: DataFrame with columns ['high', 'low', 'close', 'volume']
    :param short_period: The short period EMA for Klinger calculation
    :param long_period: The long period EMA for Klinger calculation
    :return: DataFrame with additional columns ['Klinger Oscillator', 'Klinger Signal']
    """
    # Calculate the daily range and trend
    dm = df['high'] - df['low']
    trend = df['close'] - df['close'].shift(1)
    
    # Calculate raw values for Klinger Volume Oscillator (KVO)
    cm = dm * ((df['close'] - (df['high'] + df['low']) / 2) / dm) * df['volume']
    cm = cm.fillna(0)  # Replace NaN values with zero

    # Calculate short-term and long-term EMAs of CM
    short_ema_cm = cm.ewm(span=short_period, adjust=False).mean()
    long_ema_cm = cm.ewm(span=long_period, adjust=False).mean()

    # Klinger Oscillator
    df['Klinger Oscillator'] = short_ema_cm - long_ema_cm

    # Signal line (13-period EMA of Klinger Oscillator)
    df['Klinger Signal'] = df['Klinger Oscillator'].ewm(span=13, adjust=False).mean()

    # Generate trading signals based on Klinger Oscillator
    df['Signal'] = 0
    df.loc[df['Klinger Oscillator'] > df['Klinger Signal'], 'Signal'] = 1  # Potential buy signal
    df.loc[df['Klinger Oscillator'] < df['Klinger Signal'], 'Signal'] = -1 # Potential sell signal

    return df

# Calculate Klinger Oscillator for the sample data
klinger_df = calculate_klinger_oscillator(df_1h)
correlation_results = calculate_correlation(klinger_df)
print(correlation_results)

klinger_df



{'Spearman Correlation': '-2.8379%', 'Spearman P-value': '9.568932993375901e-05%', 'Kendall Correlation': '-2.3170%', 'Kendall P-value': '9.62032152284919e-05%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,Ultimate Oscillator,TSI,DPO,PPO,Signal Line,PROC,RVI,RVI Signal,Klinger Oscillator,Klinger Signal
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,NaN,NaN,NaN,-0.192536,-0.038507,NaN,NaN,NaN,-1145.121110,-163.588730
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,NaN,NaN,NaN,-0.266355,-0.084077,NaN,NaN,NaN,-572.227056,-221.965634
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,NaN,NaN,NaN,-0.273922,-0.122046,NaN,NaN,NaN,-608.471309,-277.180730
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,NaN,NaN,NaN,-0.163563,-0.130349,NaN,NaN,NaN,662.242203,-142.977454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,0.585379,5.932767,NaN,0.955580,0.494687,2.644655,0.288866,0.217017,46346.298942,43360.549072
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,0.580672,6.942538,NaN,1.059104,0.607570,4.563801,0.308502,0.265299,49894.760643,44294.007868
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,0.604634,8.998241,NaN,1.161675,0.718391,4.623390,0.317375,0.294668,39978.183243,43677.461493


### Chande's Momentum Oscillator



In [26]:
def calculate_chandes_momentum_oscillator(df, n=14):
    """
    Calculate Chande's Momentum Oscillator for given OHLCV data.
    It helps traders identify trend strength and potential reversals.

    :param df: DataFrame with column ['close']
    :param n: The number of periods to use in the calculation
    :return: DataFrame with an additional column ['CMO']
    """
    # Calculate the difference in closing prices
    delta_close = df['close'].diff()

    # Sum of gains and losses
    sum_gains = delta_close.where(delta_close > 0, 0).rolling(window=n).sum()
    sum_losses = -delta_close.where(delta_close < 0, 0).rolling(window=n).sum()

    # Chande's Momentum Oscillator
    df['CMO'] = 100 * (sum_gains - sum_losses) / (sum_gains + sum_losses)

    # Generate trading signals based on CMO
    overbought = 50   # Example threshold, can be adjusted
    oversold = -50    # Example threshold, can be adjusted
    df['Signal'] = 0
    df.loc[df['CMO'] > overbought, 'Signal'] = -1  # Potential sell signal
    df.loc[df['CMO'] < oversold, 'Signal'] = 1    # Potential buy signal

    return df

# Calculate Chande's Momentum Oscillator for the sample data
cmo_df = calculate_chandes_momentum_oscillator(df_1h)

correlation_results = calculate_correlation(cmo_df)
print(correlation_results)

cmo_df


{'Spearman Correlation': '1.0600%', 'Spearman P-value': '6.723933939833067%', 'Kendall Correlation': '0.8632%', 'Kendall P-value': '6.348821703759322%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,TSI,DPO,PPO,Signal Line,PROC,RVI,RVI Signal,Klinger Oscillator,Klinger Signal,CMO
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,NaN,NaN,-0.192536,-0.038507,NaN,NaN,NaN,-1145.121110,-163.588730,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,NaN,NaN,-0.266355,-0.084077,NaN,NaN,NaN,-572.227056,-221.965634,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,NaN,NaN,-0.273922,-0.122046,NaN,NaN,NaN,-608.471309,-277.180730,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,NaN,NaN,-0.163563,-0.130349,NaN,NaN,NaN,662.242203,-142.977454,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,5.932767,NaN,0.955580,0.494687,2.644655,0.288866,0.217017,46346.298942,43360.549072,33.144404
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,6.942538,NaN,1.059104,0.607570,4.563801,0.308502,0.265299,49894.760643,44294.007868,57.097134
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,8.998241,NaN,1.161675,0.718391,4.623390,0.317375,0.294668,39978.183243,43677.461493,62.281180


### Schaff Trend Cycle

In [27]:
def calculate_schaff_trend_cycle(df, short_period=23, long_period=50):
    """
    Calculate the Schaff Trend Cycle (STC) for given OHLCV data.
    Schaff Trend Cycle smooths and signals trends, aiding traders in trend analysis.

    :param df: DataFrame with column ['close']
    :param short_period: The short period for MACD calculation
    :param long_period: The long period for MACD calculation
    :return: DataFrame with an additional column ['STC']
    """
    # Calculate MACD
    macd = df['close'].ewm(span=short_period, adjust=False).mean() - df['close'].ewm(span=long_period, adjust=False).mean()

    # Calculate stochastic of MACD
    stoch_macd = 100 * (macd - macd.rolling(window=long_period).min()) / (macd.rolling(window=long_period).max() - macd.rolling(window=long_period).min())

    # Calculate Schaff Trend Cycle
    df['STC'] = stoch_macd.ewm(span=short_period, adjust=False).mean()

    # Generate trading signals based on STC
    overbought = 75   # Example threshold, can be adjusted
    oversold = 25     # Example threshold, can be adjusted
    df['Signal'] = 0
    df.loc[df['STC'] > overbought, 'Signal'] = -1  # Potential sell signal
    df.loc[df['STC'] < oversold, 'Signal'] = 1    # Potential buy signal

    return df

# Calculate Schaff Trend Cycle for the sample data
stc_df = calculate_schaff_trend_cycle(df_1h)




correlation_results = calculate_correlation(stc_df)
print(correlation_results)

stc_df


{'Spearman Correlation': '-0.0931%', 'Spearman P-value': '87.23149037496223%', 'Kendall Correlation': '-0.0715%', 'Kendall P-value': '87.26044942636008%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,DPO,PPO,Signal Line,PROC,RVI,RVI Signal,Klinger Oscillator,Klinger Signal,CMO,STC
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,NaN,-0.192536,-0.038507,NaN,NaN,NaN,-1145.121110,-163.588730,NaN,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,NaN,-0.266355,-0.084077,NaN,NaN,NaN,-572.227056,-221.965634,NaN,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,NaN,-0.273922,-0.122046,NaN,NaN,NaN,-608.471309,-277.180730,NaN,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,NaN,-0.163563,-0.130349,NaN,NaN,NaN,662.242203,-142.977454,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,NaN,0.955580,0.494687,2.644655,0.288866,0.217017,46346.298942,43360.549072,33.144404,88.610200
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,NaN,1.059104,0.607570,4.563801,0.308502,0.265299,49894.760643,44294.007868,57.097134,89.559350
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,NaN,1.161675,0.718391,4.623390,0.317375,0.294668,39978.183243,43677.461493,62.281180,90.429404


## Volatility Indicators:

### List of major indicators

1. Bollinger Bands: Bollinger Bands measure price volatility, helping traders identify potential breakout or reversal points.

2. Average True Range (ATR): ATR gauges market volatility, assisting traders in setting stop-loss levels. 

3. Standard Deviation: Standard Deviation quantifies price variability, aiding traders in assessing market stability.

4. Donchian Channels: Donchian Channels identify price extremes, helping traders spot potential trend changes.

5. Volatility Smile: Volatility Smile is used to analyze options pricing, providing insights into market expectations.

6. Volatility Stop: Volatility Stop sets stop-loss levels based on market volatility, helping protect profits.

7. Volatility Squeeze: Volatility Squeeze indicates potential price breakout when market volatility decreases.

8. Volatility Ratio: Volatility Ratio measures current volatility levels, assisting traders in assessing market conditions.

9. Volatility Quality Index (VQI): VQI evaluates market noise, aiding traders in filtering out irrelevant price movements.

Indicators 4-9 are not independent indicators, they are used for risk management

### Bollinger Bands

In [28]:
def calculate_bollinger_bands(df, n=20, num_std_dev=2):
    """
    Calculate Bollinger Bands for given OHLCV data.
    Bollinger Bands measure price volatility, helping traders identify potential breakout or reversal points.

    :param df: DataFrame with column ['close']
    :param n: The number of periods to use in the calculation
    :param num_std_dev: The number of standard deviations to determine the upper and lower bands
    :return: DataFrame with additional columns ['Bollinger Upper Band', 'Bollinger Lower Band']
    """
    # Calculate the moving average
    ma = df['close'].rolling(window=n).mean()

    # Calculate the standard deviation
    std_dev = df['close'].rolling(window=n).std()

    # Calculate upper and lower Bollinger Bands
    df['Bollinger Upper Band'] = ma + (std_dev * num_std_dev)
    df['Bollinger Lower Band'] = ma - (std_dev * num_std_dev)

    # Generate trading signals based on Bollinger Bands
    df['Signal'] = 0
    df.loc[df['close'] > df['Bollinger Upper Band'], 'Signal'] = -1  # Potential sell signal
    df.loc[df['close'] < df['Bollinger Lower Band'], 'Signal'] = 1   # Potential buy signal

    return df

# Calculate Bollinger Bands for the sample data
bollinger_df = calculate_bollinger_bands(df_1h)
correlation_results = calculate_correlation(bollinger_df)
print(correlation_results)

bollinger_df



{'Spearman Correlation': '3.3781%', 'Spearman P-value': '5.424385694841972e-07%', 'Kendall Correlation': '2.7463%', 'Kendall P-value': '3.9096715811519317e-07%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,Signal Line,PROC,RVI,RVI Signal,Klinger Oscillator,Klinger Signal,CMO,STC,Bollinger Upper Band,Bollinger Lower Band
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,-0.038507,NaN,NaN,NaN,-1145.121110,-163.588730,NaN,NaN,NaN,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,-0.084077,NaN,NaN,NaN,-572.227056,-221.965634,NaN,NaN,NaN,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,-0.122046,NaN,NaN,NaN,-608.471309,-277.180730,NaN,NaN,NaN,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,-0.130349,NaN,NaN,NaN,662.242203,-142.977454,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,0.494687,2.644655,0.288866,0.217017,46346.298942,43360.549072,33.144404,88.610200,37391.354636,34055.678364
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,0.607570,4.563801,0.308502,0.265299,49894.760643,44294.007868,57.097134,89.559350,37519.707847,34015.682153
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,0.718391,4.623390,0.317375,0.294668,39978.183243,43677.461493,62.281180,90.429404,37671.518300,33970.595700


### Average True Range (ATR)

In [29]:
def calculate_average_true_range(df, n=14):
    """
    Calculate the Average True Range (ATR) for given OHLCV data.
    ATR gauges market volatility, assisting traders in setting stop-loss levels.

    :param df: DataFrame with columns ['high', 'low', 'close']
    :param n: The number of periods to use in the ATR calculation
    :return: DataFrame with an additional column ['ATR']
    """
    # Calculate True Range (TR)
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift())
    low_close = abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    # Calculate Average True Range
    df['ATR'] = tr.rolling(window=n).mean()

    return df

# Calculate Average True Range for the sample data
atr_df = calculate_average_true_range(df_1h)
correlation_results = calculate_correlation(atr_df)
print(correlation_results)
atr_df



{'Spearman Correlation': '3.3781%', 'Spearman P-value': '5.424385694841972e-07%', 'Kendall Correlation': '2.7463%', 'Kendall P-value': '3.9096715811519317e-07%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,PROC,RVI,RVI Signal,Klinger Oscillator,Klinger Signal,CMO,STC,Bollinger Upper Band,Bollinger Lower Band,ATR
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,NaN,NaN,NaN,-1145.121110,-163.588730,NaN,NaN,NaN,NaN,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,NaN,NaN,NaN,-572.227056,-221.965634,NaN,NaN,NaN,NaN,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,NaN,NaN,NaN,-608.471309,-277.180730,NaN,NaN,NaN,NaN,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,NaN,NaN,NaN,662.242203,-142.977454,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,2.644655,0.288866,0.217017,46346.298942,43360.549072,33.144404,88.610200,37391.354636,34055.678364,634.342143
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,4.563801,0.308502,0.265299,49894.760643,44294.007868,57.097134,89.559350,37519.707847,34015.682153,624.590000
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,4.623390,0.317375,0.294668,39978.183243,43677.461493,62.281180,90.429404,37671.518300,33970.595700,630.277857


### Standard Deviation

In [30]:
def calculate_standard_deviation(df, n=20):
    """
    Calculate the Standard Deviation for given OHLCV data.
    Standard Deviation quantifies price variability, aiding traders in assessing market stability.

    :param df: DataFrame with column ['close']
    :param n: The number of periods to use in the Standard Deviation calculation
    :return: DataFrame with an additional column ['Standard Deviation']
    """
    df['Standard Deviation'] = df['close'].rolling(window=n).std()

    return df

# Calculate Standard Deviation for the sample data
std_dev_df = calculate_standard_deviation(df_1h)
correlation_results = calculate_correlation(std_dev_df)
print(correlation_results)
std_dev_df



{'Spearman Correlation': '3.3781%', 'Spearman P-value': '5.424385694841972e-07%', 'Kendall Correlation': '2.7463%', 'Kendall P-value': '3.9096715811519317e-07%'}


,open,high,low,close,volume,return,return_2,return_abs,return_abs_2,%K,...,RVI,RVI Signal,Klinger Oscillator,Klinger Signal,CMO,STC,Bollinger Upper Band,Bollinger Lower Band,ATR,Standard Deviation
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,-0.024093,-0.014697,0.024093,0.014697,NaN,...,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0.009628,0.015676,0.009628,0.015676,NaN,...,NaN,NaN,-1145.121110,-163.588730,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0.005990,0.020317,0.005990,0.020317,NaN,...,NaN,NaN,-572.227056,-221.965634,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0.014242,0.011108,0.014242,0.011108,NaN,...,NaN,NaN,-608.471309,-277.180730,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,-0.003089,0.013190,0.003089,0.013190,NaN,...,NaN,NaN,662.242203,-142.977454,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 19:30:00,36721.29,36853.94,36333.44,36620.39,4333.305783,0.009305,0.013568,0.009305,0.013568,78.950064,...,0.288866,0.217017,46346.298942,43360.549072,33.144404,88.610200,37391.354636,34055.678364,634.342143,833.919068
2021-05-31 20:30:00,36620.39,37149.99,36580.01,36961.16,4167.824255,0.004223,-0.000715,0.004223,0.000715,89.857531,...,0.308502,0.265299,49894.760643,44294.007868,57.097134,89.559350,37519.707847,34015.682153,624.590000,876.006423
2021-05-31 21:30:00,36961.16,37457.07,36843.24,37117.24,6313.228329,-0.004917,-0.011396,0.004917,0.011396,89.712191,...,0.317375,0.294668,39978.183243,43677.461493,62.281180,90.429404,37671.518300,33970.595700,630.277857,925.230650


## Using TA-Lib


Implementing indicators using TA-Lib and measuring it performance with correlation against future returns


#### Volume Indicators
1. AD - Chaikin A/D Line
2. ADOSC - Chaikin A/D Oscillator
3. OBV - On Balance Volume


#### Momentum Indicators

1. ADX - Average Directional Movement Index
2. ADXR - Average Directional Movement Index Rating
3. APO - Absolute Price Oscillator
4. AROON - Aroon
5. AROONOSC - Aroon Oscillator
6. BOP - Balance Of Power
7. CCI - Commodity Channel Index
8. CMO - Chande Momentum Oscillator
9. DX - Directional Movement Index
10. MACD - Moving Average Convergence/Divergence
11. MACDEXT - MACD with controllable MA type
12. MACDFIX - Moving Average Convergence/Divergence Fix 12/26
13. MFI - Money Flow Index
14. MINUS_DI - Minus Directional Indicator
15. MINUS_DM - Minus Directional Movement
16. MOM - Momentum
17. PLUS_DI - Plus Directional Indicator
18. PLUS_DM - Plus Directional Movement
19. PPO - Percentage Price Oscillator
20. ROC - Rate of change : ((price/prevPrice)-1)*100
21. ROCP - Rate of change Percentage: (price-prevPrice)/prevPrice
22. ROCR - Rate of change ratio: (price/prevPrice)
23. ROCR100 - Rate of change ratio 100 scale: (price/prevPrice)*100
24. RSI - Relative Strength Index
25. STOCH - Stochastic
26. STOCHF - Stochastic Fast
27. STOCHRSI - Stochastic Relative Strength Index
28. TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
29. ULTOSC - Ultimate Oscillator
30. WILLR - Williams' %R


Pattern Recognition

1. CDL2CROWS - Two Crows
2. CDL3BLACKCROWS - Three Black Crows
3. CDL3INSIDE - Three Inside Up/Down
4. CDL3LINESTRIKE - Three-Line Strike 
5. CDL3OUTSIDE - Three Outside Up/Down
6. CDL3STARSINSOUTH - Three Stars In The South
7. CDL3WHITESOLDIERS - Three Advancing White Soldiers
8. CDLABANDONEDBABY - Abandoned Baby
9. CDLADVANCEBLOCK - Advance Block
10. CDLBELTHOLD - Belt-hold
11. CDLBREAKAWAY - Breakaway
12. CDLCLOSINGMARUBOZU - Closing Marubozu
13. CDLCONCEALBABYSWALL - Concealing Baby Swallow
14. CDLCOUNTERATTACK - Counterattack
15. CDLDARKCLOUDCOVER - Dark Cloud Cover
16. CDLDOJI - Doji
17. CDLDOJISTAR - Doji Star
18. CDLDRAGONFLYDOJI - Dragonfly Doji
19. CDLENGULFING - Engulfing Pattern
20. CDLEVENINGDOJISTAR - Evening Doji Star
21. CDLEVENINGSTAR - Evening Star
22. CDLGAPSIDESIDEWHITE - Up/Down-gap side-by-side white lines
23. CDLGRAVESTONEDOJI - Gravestone Doji
24. CDLHAMMER - Hammer
25. CDLHANGINGMAN - Hanging Man
26. CDLHARAMI - Harami Pattern
27. CDLHARAMICROSS - Harami Cross Pattern
28. CDLHIGHWAVE - High-Wave Candle
29. CDLHIKKAKE - Hikkake Pattern
30. CDLHIKKAKEMOD - Modified Hikkake Pattern
31. CDLHOMINGPIGEON - Homing Pigeon
32. CDLIDENTICAL3CROWS - Identical Three Crows
33. CDLINNECK - In-Neck Pattern
And many more
<!-- CDLINVERTEDHAMMER - Inverted Hammer

CDLKICKING - Kicking
CDLKICKINGBYLENGTH - Kicking - bull/bear determined by the longer marubozu
CDLLADDERBOTTOM - Ladder Bottom
CDLLONGLEGGEDDOJI - Long Legged Doji
CDLLONGLINE - Long Line Candle
CDLMARUBOZU - Marubozu
CDLMATCHINGLOW - Matching Low
CDLMATHOLD - Mat Hold
CDLMORNINGDOJISTAR - Morning Doji Star
CDLMORNINGSTAR - Morning Star
CDLONNECK - On-Neck Pattern
CDLPIERCING - Piercing Pattern
CDLRICKSHAWMAN - Rickshaw Man
CDLRISEFALL3METHODS - Rising/Falling Three Methods
CDLSEPARATINGLINES - Separating Lines
CDLSHOOTINGSTAR - Shooting Star
CDLSHORTLINE - Short Line Candle
CDLSPINNINGTOP - Spinning Top
CDLSTALLEDPATTERN - Stalled Pattern
CDLSTICKSANDWICH - Stick Sandwich
CDLTAKURI - Takuri (Dragonfly Doji with very long lower shadow)
CDLTASUKIGAP - Tasuki Gap
CDLTHRUSTING - Thrusting Pattern
CDLTRISTAR - Tristar Pattern
CDLUNIQUE3RIVER - Unique 3 River
CDLUPSIDEGAP2CROWS - Upside Gap Two Crows
CDLXSIDEGAP3METHODS - Upside/Downside Gap Three Methods -->


### Results of correlation with Return_1 (t+1/t) and return_2(t+2/2) are fairly similar
### Sensitivity wrt Tick sizes aren't significant for 3m,5m some indicators show up at higher sizes 30m,1h : Details shared in the excel
### Rest results are compiled in Excel

In [31]:

df_5mn = df_5m.copy()

# Accumulation/Distribution Line (A/D)
df_5mn['AD'] = talib.AD(df_5mn['high'], df_5mn['low'], df_5mn['close'], df_5mn['volume'])
df_5mn['ADX'] = talib.ADX(df_5mn['high'], df_5mn['low'], df_5mn['close'], timeperiod=14)
df_5mn['ADXR'] = talib.ADXR(df_5mn['high'], df_5mn['low'], df_5mn['close'], timeperiod=14)
df_5mn['APO'] = talib.APO(df_5mn['close'], fastperiod=12, slowperiod=26)
df_5mn['AROON_UP'], df_5mn['AROON_DOWN'] = talib.AROON(df_5mn['high'], df_5mn['low'], timeperiod=14)
df_5mn['AROONOSC'] = talib.AROONOSC(df_5mn['high'], df_5mn['low'], timeperiod=14)
df_5mn['BOP'] = talib.BOP(df_5mn['open'], df_5mn['high'], df_5mn['low'], df_5mn['close'])
df_5mn['CCI'] = talib.CCI(df_5mn['high'], df_5mn['low'], df_5mn['close'], timeperiod=14)
df_5mn['CMO'] = talib.CMO(df_5mn['close'], timeperiod=14)
df_5mn['DX'] = talib.DX(df_5mn['high'], df_5mn['low'], df_5mn['close'], timeperiod=14)
df_5mn['MACD'], df_5mn['MACDSIGNAL'], _ = talib.MACD(df_5mn['close'], fastperiod=12, slowperiod=26, signalperiod=9)
df_5mn['MACDEXT'], df_5mn['MACDSIGNALEXT'], _ = talib.MACDEXT(df_5mn['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
df_5mn['MACDFIX'], df_5mn['MACDSIGNALFIX'], _ = talib.MACDFIX(df_5mn['close'], signalperiod=9)
df_5mn['MFI'] = talib.MFI(df_5mn['high'], df_5mn['low'], df_5mn['close'], df_5mn['volume'], timeperiod=14)
df_5mn['MINUS_DI'] = talib.MINUS_DI(df_5mn['high'], df_5mn['low'], df_5mn['close'], timeperiod=14)
df_5mn['MINUS_DM'] = talib.MINUS_DM(df_5mn['high'], df_5mn['low'], timeperiod=14)
df_5mn['MOM'] = talib.MOM(df_5mn['close'], timeperiod=10)
df_5mn['OBV'] = talib.OBV(df_5mn['close'], df_5mn['volume'])
df_5mn['PLUS_DI'] = talib.PLUS_DI(df_5mn['high'], df_5mn['low'], df_5mn['close'], timeperiod=14)
df_5mn['PLUS_DM'] = talib.PLUS_DM(df_5mn['high'], df_5mn['low'], timeperiod=14)
df_5mn['PPO'] = talib.PPO(df_5mn['close'], fastperiod=12, slowperiod=26)
df_5mn['ROC'] = talib.ROC(df_5mn['close'], timeperiod=10)
df_5mn['ROCP'] = talib.ROCP(df_5mn['close'], timeperiod=10)
df_5mn['ROCR'] = talib.ROCR(df_5mn['close'], timeperiod=10)
df_5mn['ROCR100'] = talib.ROCR100(df_5mn['close'], timeperiod=10)
df_5mn['RSI'] = talib.RSI(df_5mn['close'], timeperiod=14)
df_5mn['STOCH_K'], df_5mn['STOCH_D'] = talib.STOCH(df_5mn['high'], df_5mn['low'], df_5mn['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_5mn['STOCHF_K'], df_5mn['STOCHF_D'] = talib.STOCHF(df_5mn['high'], df_5mn['low'], df_5mn['close'], fastk_period=5, fastd_period=3, fastd_matype=0)
df_5mn['STOCHRSI_K'], df_5mn['STOCHRSI_D'] = talib.STOCHRSI(df_5mn['close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
df_5mn['TRIX'] = talib.TRIX(df_5mn['close'], timeperiod=30)
df_5mn['ULTOSC'] = talib.ULTOSC(df_5mn['high'], df_5mn['low'], df_5mn['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)
df_5mn['WILLR'] = talib.WILLR(df_5mn['high'], df_5mn['low'], df_5mn['close'], timeperiod=14)
df_5mn.dropna(inplace=True)


In [32]:
df_3mn = df_3m.copy()
# Replace df_5mn with df_3mn
df_3mn['AD'] = talib.AD(df_3mn['high'], df_3mn['low'], df_3mn['close'], df_3mn['volume'])
df_3mn['ADX'] = talib.ADX(df_3mn['high'], df_3mn['low'], df_3mn['close'], timeperiod=14)
df_3mn['ADXR'] = talib.ADXR(df_3mn['high'], df_3mn['low'], df_3mn['close'], timeperiod=14)
df_3mn['APO'] = talib.APO(df_3mn['close'], fastperiod=12, slowperiod=26)
df_3mn['AROON_UP'], df_3mn['AROON_DOWN'] = talib.AROON(df_3mn['high'], df_3mn['low'], timeperiod=14)
df_3mn['AROONOSC'] = talib.AROONOSC(df_3mn['high'], df_3mn['low'], timeperiod=14)
df_3mn['BOP'] = talib.BOP(df_3mn['open'], df_3mn['high'], df_3mn['low'], df_3mn['close'])
df_3mn['CCI'] = talib.CCI(df_3mn['high'], df_3mn['low'], df_3mn['close'], timeperiod=14)
df_3mn['CMO'] = talib.CMO(df_3mn['close'], timeperiod=14)
df_3mn['DX'] = talib.DX(df_3mn['high'], df_3mn['low'], df_3mn['close'], timeperiod=14)
df_3mn['MACD'], df_3mn['MACDSIGNAL'], _ = talib.MACD(df_3mn['close'], fastperiod=12, slowperiod=26, signalperiod=9)
df_3mn['MACDEXT'], df_3mn['MACDSIGNALEXT'], _ = talib.MACDEXT(df_3mn['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
df_3mn['MACDFIX'], df_3mn['MACDSIGNALFIX'], _ = talib.MACDFIX(df_3mn['close'], signalperiod=9)
df_3mn['MFI'] = talib.MFI(df_3mn['high'], df_3mn['low'], df_3mn['close'], df_3mn['volume'], timeperiod=14)
df_3mn['MINUS_DI'] = talib.MINUS_DI(df_3mn['high'], df_3mn['low'], df_3mn['close'], timeperiod=14)
df_3mn['MINUS_DM'] = talib.MINUS_DM(df_3mn['high'], df_3mn['low'], timeperiod=14)
df_3mn['MOM'] = talib.MOM(df_3mn['close'], timeperiod=10)
df_3mn['OBV'] = talib.OBV(df_3mn['close'], df_3mn['volume'])
df_3mn['PLUS_DI'] = talib.PLUS_DI(df_3mn['high'], df_3mn['low'], df_3mn['close'], timeperiod=14)
df_3mn['PLUS_DM'] = talib.PLUS_DM(df_3mn['high'], df_3mn['low'], timeperiod=14)
df_3mn['PPO'] = talib.PPO(df_3mn['close'], fastperiod=12, slowperiod=26)
df_3mn['ROC'] = talib.ROC(df_3mn['close'], timeperiod=10)
df_3mn['ROCP'] = talib.ROCP(df_3mn['close'], timeperiod=10)
df_3mn['ROCR'] = talib.ROCR(df_3mn['close'], timeperiod=10)
df_3mn['ROCR100'] = talib.ROCR100(df_3mn['close'], timeperiod=10)
df_3mn['RSI'] = talib.RSI(df_3mn['close'], timeperiod=14)
df_3mn['STOCH_K'], df_3mn['STOCH_D'] = talib.STOCH(df_3mn['high'], df_3mn['low'], df_3mn['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_3mn['STOCHF_K'], df_3mn['STOCHF_D'] = talib.STOCHF(df_3mn['high'], df_3mn['low'], df_3mn['close'], fastk_period=5, fastd_period=3, fastd_matype=0)
df_3mn['STOCHRSI_K'], df_3mn['STOCHRSI_D'] = talib.STOCHRSI(df_3mn['close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
df_3mn['TRIX'] = talib.TRIX(df_3mn['close'], timeperiod=30)
df_3mn['ULTOSC'] = talib.ULTOSC(df_3mn['high'], df_3mn['low'], df_3mn['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)
df_3mn['WILLR'] = talib.WILLR(df_3mn['high'], df_3mn['low'], df_3mn['close'], timeperiod=14)
df_3mn.dropna(inplace=True)


In [33]:
df_15mn=df_15m.copy()
df_15mn['AD'] = talib.AD(df_15mn['high'], df_15mn['low'], df_15mn['close'], df_15mn['volume'])
df_15mn['ADX'] = talib.ADX(df_15mn['high'], df_15mn['low'], df_15mn['close'], timeperiod=14)
df_15mn['ADXR'] = talib.ADXR(df_15mn['high'], df_15mn['low'], df_15mn['close'], timeperiod=14)
df_15mn['APO'] = talib.APO(df_15mn['close'], fastperiod=12, slowperiod=26)
df_15mn['AROON_UP'], df_15mn['AROON_DOWN'] = talib.AROON(df_15mn['high'], df_15mn['low'], timeperiod=14)
df_15mn['AROONOSC'] = talib.AROONOSC(df_15mn['high'], df_15mn['low'], timeperiod=14)
df_15mn['BOP'] = talib.BOP(df_15mn['open'], df_15mn['high'], df_15mn['low'], df_15mn['close'])
df_15mn['CCI'] = talib.CCI(df_15mn['high'], df_15mn['low'], df_15mn['close'], timeperiod=14)
df_15mn['CMO'] = talib.CMO(df_15mn['close'], timeperiod=14)
df_15mn['DX'] = talib.DX(df_15mn['high'], df_15mn['low'], df_15mn['close'], timeperiod=14)
df_15mn['MACD'], df_15mn['MACDSIGNAL'], _ = talib.MACD(df_15mn['close'], fastperiod=12, slowperiod=26, signalperiod=9)
df_15mn['MACDEXT'], df_15mn['MACDSIGNALEXT'], _ = talib.MACDEXT(df_15mn['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
df_15mn['MACDFIX'], df_15mn['MACDSIGNALFIX'], _ = talib.MACDFIX(df_15mn['close'], signalperiod=9)
df_15mn['MFI'] = talib.MFI(df_15mn['high'], df_15mn['low'], df_15mn['close'], df_15mn['volume'], timeperiod=14)
df_15mn['MINUS_DI'] = talib.MINUS_DI(df_15mn['high'], df_15mn['low'], df_15mn['close'], timeperiod=14)
df_15mn['MINUS_DM'] = talib.MINUS_DM(df_15mn['high'], df_15mn['low'], timeperiod=14)
df_15mn['MOM'] = talib.MOM(df_15mn['close'], timeperiod=10)
df_15mn['OBV'] = talib.OBV(df_15mn['close'], df_15mn['volume'])
df_15mn['PLUS_DI'] = talib.PLUS_DI(df_15mn['high'], df_15mn['low'], df_15mn['close'], timeperiod=14)
df_15mn['PLUS_DM'] = talib.PLUS_DM(df_15mn['high'], df_15mn['low'], timeperiod=14)
df_15mn['PPO'] = talib.PPO(df_15mn['close'], fastperiod=12, slowperiod=26)
df_15mn['ROC'] = talib.ROC(df_15mn['close'], timeperiod=10)
df_15mn['ROCP'] = talib.ROCP(df_15mn['close'], timeperiod=10)
df_15mn['ROCR'] = talib.ROCR(df_15mn['close'], timeperiod=10)
df_15mn['ROCR100'] = talib.ROCR100(df_15mn['close'], timeperiod=10)
df_15mn['RSI'] = talib.RSI(df_15mn['close'], timeperiod=14)
df_15mn['STOCH_K'], df_15mn['STOCH_D'] = talib.STOCH(df_15mn['high'], df_15mn['low'], df_15mn['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_15mn['STOCHF_K'], df_15mn['STOCHF_D'] = talib.STOCHF(df_15mn['high'], df_15mn['low'], df_15mn['close'], fastk_period=5, fastd_period=3, fastd_matype=0)
df_15mn['STOCHRSI_K'], df_15mn['STOCHRSI_D'] = talib.STOCHRSI(df_15mn['close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
df_15mn['TRIX'] = talib.TRIX(df_15mn['close'], timeperiod=30)
df_15mn['ULTOSC'] = talib.ULTOSC(df_15mn['high'], df_15mn['low'], df_15mn['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)
df_15mn['WILLR'] = talib.WILLR(df_15mn['high'], df_15mn['low'], df_15mn['close'], timeperiod=14)
df_15mn.dropna(inplace=True)

In [34]:
df_30mn=df_30m.copy()
df_30mn['AD'] = talib.AD(df_30mn['high'], df_30mn['low'], df_30mn['close'], df_30mn['volume'])
df_30mn['ADX'] = talib.ADX(df_30mn['high'], df_30mn['low'], df_30mn['close'], timeperiod=14)
df_30mn['ADXR'] = talib.ADXR(df_30mn['high'], df_30mn['low'], df_30mn['close'], timeperiod=14)
df_30mn['APO'] = talib.APO(df_30mn['close'], fastperiod=12, slowperiod=26)
df_30mn['AROON_UP'], df_30mn['AROON_DOWN'] = talib.AROON(df_30mn['high'], df_30mn['low'], timeperiod=14)
df_30mn['AROONOSC'] = talib.AROONOSC(df_30mn['high'], df_30mn['low'], timeperiod=14)
df_30mn['BOP'] = talib.BOP(df_30mn['open'], df_30mn['high'], df_30mn['low'], df_30mn['close'])
df_30mn['CCI'] = talib.CCI(df_30mn['high'], df_30mn['low'], df_30mn['close'], timeperiod=14)
df_30mn['CMO'] = talib.CMO(df_30mn['close'], timeperiod=14)
df_30mn['DX'] = talib.DX(df_30mn['high'], df_30mn['low'], df_30mn['close'], timeperiod=14)
df_30mn['MACD'], df_30mn['MACDSIGNAL'], _ = talib.MACD(df_30mn['close'], fastperiod=12, slowperiod=26, signalperiod=9)
df_30mn['MACDEXT'], df_30mn['MACDSIGNALEXT'], _ = talib.MACDEXT(df_30mn['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
df_30mn['MACDFIX'], df_30mn['MACDSIGNALFIX'], _ = talib.MACDFIX(df_30mn['close'], signalperiod=9)
df_30mn['MFI'] = talib.MFI(df_30mn['high'], df_30mn['low'], df_30mn['close'], df_30mn['volume'], timeperiod=14)
df_30mn['MINUS_DI'] = talib.MINUS_DI(df_30mn['high'], df_30mn['low'], df_30mn['close'], timeperiod=14)
df_30mn['MINUS_DM'] = talib.MINUS_DM(df_30mn['high'], df_30mn['low'], timeperiod=14)
df_30mn['MOM'] = talib.MOM(df_30mn['close'], timeperiod=10)
df_30mn['OBV'] = talib.OBV(df_30mn['close'], df_30mn['volume'])
df_30mn['PLUS_DI'] = talib.PLUS_DI(df_30mn['high'], df_30mn['low'], df_30mn['close'], timeperiod=14)
df_30mn['PLUS_DM'] = talib.PLUS_DM(df_30mn['high'], df_30mn['low'], timeperiod=14)
df_30mn['PPO'] = talib.PPO(df_30mn['close'], fastperiod=12, slowperiod=26)
df_30mn['ROC'] = talib.ROC(df_30mn['close'], timeperiod=10)
df_30mn['ROCP'] = talib.ROCP(df_30mn['close'], timeperiod=10)
df_30mn['ROCR'] = talib.ROCR(df_30mn['close'], timeperiod=10)
df_30mn['ROCR100'] = talib.ROCR100(df_30mn['close'], timeperiod=10)
df_30mn['RSI'] = talib.RSI(df_30mn['close'], timeperiod=14)
df_30mn['STOCH_K'], df_30mn['STOCH_D'] = talib.STOCH(df_30mn['high'], df_30mn['low'], df_30mn['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_30mn['STOCHF_K'], df_30mn['STOCHF_D'] = talib.STOCHF(df_30mn['high'], df_30mn['low'], df_30mn['close'], fastk_period=5, fastd_period=3, fastd_matype=0)
df_30mn['STOCHRSI_K'], df_30mn['STOCHRSI_D'] = talib.STOCHRSI(df_30mn['close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
df_30mn['TRIX'] = talib.TRIX(df_30mn['close'], timeperiod=30)
df_30mn['ULTOSC'] = talib.ULTOSC(df_30mn['high'], df_30mn['low'], df_30mn['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)
df_30mn['WILLR'] = talib.WILLR(df_30mn['high'], df_30mn['low'], df_30mn['close'], timeperiod=14)
df_30mn.dropna(inplace=True)


In [35]:
df_1hrn=df_1h.copy()
# Rewriting the code for the df_1hr dataframe

df_1hrn['AD'] = talib.AD(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], df_1hrn['volume'])
df_1hrn['ADX'] = talib.ADX(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], timeperiod=14)
df_1hrn['ADXR'] = talib.ADXR(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], timeperiod=14)
df_1hrn['APO'] = talib.APO(df_1hrn['close'], fastperiod=12, slowperiod=26)
df_1hrn['AROON_UP'], df_1hrn['AROON_DOWN'] = talib.AROON(df_1hrn['high'], df_1hrn['low'], timeperiod=14)
df_1hrn['AROONOSC'] = talib.AROONOSC(df_1hrn['high'], df_1hrn['low'], timeperiod=14)
df_1hrn['BOP'] = talib.BOP(df_1hrn['open'], df_1hrn['high'], df_1hrn['low'], df_1hrn['close'])
df_1hrn['CCI'] = talib.CCI(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], timeperiod=14)
df_1hrn['CMO'] = talib.CMO(df_1hrn['close'], timeperiod=14)
df_1hrn['DX'] = talib.DX(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], timeperiod=14)
df_1hrn['MACD'], df_1hrn['MACDSIGNAL'], _ = talib.MACD(df_1hrn['close'], fastperiod=12, slowperiod=26, signalperiod=9)
df_1hrn['MACDEXT'], df_1hrn['MACDSIGNALEXT'], _ = talib.MACDEXT(df_1hrn['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
df_1hrn['MACDFIX'], df_1hrn['MACDSIGNALFIX'], _ = talib.MACDFIX(df_1hrn['close'], signalperiod=9)
df_1hrn['MFI'] = talib.MFI(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], df_1hrn['volume'], timeperiod=14)
df_1hrn['MINUS_DI'] = talib.MINUS_DI(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], timeperiod=14)
df_1hrn['MINUS_DM'] = talib.MINUS_DM(df_1hrn['high'], df_1hrn['low'], timeperiod=14)
df_1hrn['MOM'] = talib.MOM(df_1hrn['close'], timeperiod=10)
df_1hrn['OBV'] = talib.OBV(df_1hrn['close'], df_1hrn['volume'])
df_1hrn['PLUS_DI'] = talib.PLUS_DI(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], timeperiod=14)
df_1hrn['PLUS_DM'] = talib.PLUS_DM(df_1hrn['high'], df_1hrn['low'], timeperiod=14)
df_1hrn['PPO'] = talib.PPO(df_1hrn['close'], fastperiod=12, slowperiod=26)
df_1hrn['ROC'] = talib.ROC(df_1hrn['close'], timeperiod=10)
df_1hrn['ROCP'] = talib.ROCP(df_1hrn['close'], timeperiod=10)
df_1hrn['ROCR'] = talib.ROCR(df_1hrn['close'], timeperiod=10)
df_1hrn['ROCR100'] = talib.ROCR100(df_1hrn['close'], timeperiod=10)
df_1hrn['RSI'] = talib.RSI(df_1hrn['close'], timeperiod=14)
df_1hrn['STOCH_K'], df_1hrn['STOCH_D'] = talib.STOCH(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_1hrn['STOCHF_K'], df_1hrn['STOCHF_D'] = talib.STOCHF(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], fastk_period=5, fastd_period=3, fastd_matype=0)
df_1hrn['STOCHRSI_K'], df_1hrn['STOCHRSI_D'] = talib.STOCHRSI(df_1hrn['close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
df_1hrn['TRIX'] = talib.TRIX(df_1hrn['close'], timeperiod=30)
df_1hrn['ULTOSC'] = talib.ULTOSC(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)
df_1hrn['WILLR'] = talib.WILLR(df_1hrn['high'], df_1hrn['low'], df_1hrn['close'], timeperiod=14)
df_1hrn.dropna(inplace=True)


In [36]:
indicators = ['AD','ADX', 'ADXR', 'APO', 'AROON_UP', 'AROON_DOWN', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'OBV','ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH_K', 'STOCH_D', 'STOCHF_K', 'STOCHF_D', 'STOCHRSI_K', 'STOCHRSI_D', 'TRIX', 'ULTOSC', 'WILLR', 'close']


##### 30 Min Data

In [37]:

# Assuming df_30mn is your DataFrame and indicators list is defined as above

# Select only the necessary columns
columns_of_interest = indicators + ['return', 'return_abs', 'return_2', 'return_2_abs']
data = df_30mn[columns_of_interest]

# Function to calculate p-values
def calculate_p_values(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(scipy.stats.pearsonr(df[r], df[c])[1], 4)
    return pvalues

# Calculate correlation matrices and their corresponding p-values
pearson_corr_30 = data.corr(method='pearson')*100
pearson_p_values_30 = calculate_p_values(data)*100

spearman_corr_30 = data.corr(method='spearman')*100
spearman_p_values_30 = calculate_p_values(data.apply(scipy.stats.rankdata))*100

kendall_corr = data.corr(method='kendall')*100
kendall_p_values = calculate_p_values(data.apply(scipy.stats.rankdata))*100



##### Pearson

In [38]:
# Extracting relevant correlations and p-values for Pearson
pearson_corr_subset_30 = pearson_corr_30.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
pearson_p_values_subset_30 = pearson_p_values_30.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]

# Combining correlation values and p-values into a single table for Pearson
pearson_table = pearson_corr_subset_30.round(4).astype(float) 
pearson_table
## Values are in percentages

,return,return_abs,return_2,return_2_abs
AD,0.5797,5.2033,0.8232,4.9074
ADX,0.0409,9.8212,0.0147,9.3039
ADXR,-0.1008,9.7665,-0.1553,9.6995
APO,-0.7134,-8.0910,-0.6966,-7.4282
AROON_UP,0.8994,6.1376,1.4382,4.7939
AROON_DOWN,-0.6016,-1.2715,-0.9100,-1.7690
AROONOSC,-0.8929,-4.3893,-1.3965,-3.8948
BOP,-3.5483,-3.2697,-2.9897,-2.7128
CCI,-0.4986,-5.1068,-0.9522,-4.4672
CMO,-0.5673,-6.9168,-0.4549,-6.3543


In [39]:
# Extracting relevant correlations and p-values for Spearman
spearman_corr_subset_30= spearman_corr_30.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
spearman_p_values_subset_30 = spearman_p_values_30.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]


# Combining correlation values and p-values into a single table for Spearman
spearman_table_30 = spearman_corr_subset_30.round(4).astype(float)
spearman_table_30

,return,return_abs,return_2,return_2_abs
AD,0.5548,0.8912,1.1299,0.8044
ADX,0.6278,9.9447,1.1672,8.8513
ADXR,0.4322,9.6175,0.9832,9.0222
APO,-0.6163,-2.8719,-1.5230,-2.8220
AROON_UP,2.1646,4.9275,3.4239,4.4917
AROON_DOWN,-2.2172,0.0113,-3.5265,-0.4533
AROONOSC,-2.6759,-2.9142,-4.3032,-2.8880
BOP,-9.6949,-2.5063,-7.8319,-1.9787
CCI,-5.6468,-3.7672,-6.8033,-3.8999
CMO,-4.5153,-5.1596,-5.2093,-5.1592


In [40]:
data_3mn = df_3mn[columns_of_interest]

# Calculate correlation matrices and their corresponding p-values
pearson_corr = data_3mn.corr(method='pearson')*100
pearson_p_values = calculate_p_values(data_3mn)*100

spearman_corr = data_3mn.corr(method='spearman')*100
spearman_p_values = calculate_p_values(data_3mn.apply(scipy.stats.rankdata))*100

kendall_corr = data_3mn.corr(method='kendall')*100
kendall_p_values = calculate_p_values(data_3mn.apply(scipy.stats.rankdata))*100

# Extracting relevant correlations and p-values for Pearson
pearson_corr_subset = pearson_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
pearson_p_values_subset = pearson_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]



In [41]:
# Combining correlation values and p-values into a single table for Pearson
pearson_table = pearson_corr_subset.round(4).astype(float)
pearson_table
## Values are in percentages

,return,return_abs,return_2,return_2_abs
AD,0.1779,5.4994,0.2560,5.6290
ADX,0.4231,13.2117,0.5866,12.1068
ADXR,0.4366,10.9115,0.6062,10.1749
APO,-1.2688,-8.4164,-1.6675,-8.0124
AROON_UP,0.8767,4.6050,1.2793,3.7620
AROON_DOWN,-0.7261,-2.0978,-1.0628,-2.5819
AROONOSC,-0.9295,-3.8826,-1.3582,-3.6775
BOP,-1.5801,-1.2048,-2.0932,-1.1370
CCI,-0.7344,-3.9503,-1.0901,-3.6358
CMO,-1.3735,-5.5675,-1.5889,-5.1442


In [42]:
# Extracting relevant correlations and p-values for Spearman
spearman_corr_subset = spearman_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
spearman_p_values_subset = spearman_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]

# Combining correlation values and p-values into a single table for Spearman
spearman_table = spearman_corr_subset.round(4).astype(float)
spearman_table

,return,return_abs,return_2,return_2_abs
AD,0.1270,4.4449,0.2695,4.2881
ADX,0.3288,13.6205,0.7015,12.4428
ADXR,0.2791,11.9927,0.6467,11.2496
APO,-1.8662,-2.2831,-2.9970,-2.1607
AROON_UP,2.8732,3.3124,4.2660,2.4329
AROON_DOWN,-2.8704,-1.5251,-4.2867,-2.1903
AROONOSC,-3.4834,-2.9035,-5.1940,-2.7544
BOP,-4.8817,-0.8116,-5.5687,-0.8171
CCI,-4.8982,-3.0801,-6.5838,-2.8633
CMO,-5.3765,-3.7259,-6.9752,-3.5438


In [43]:
data_5mn = df_5mn[columns_of_interest]

# Calculate correlation matrices and their corresponding p-values
pearson_corr = data_5mn.corr(method='pearson')*100
pearson_p_values = calculate_p_values(data_5mn)*100

spearman_corr = data_5mn.corr(method='spearman')*100
spearman_p_values = calculate_p_values(data_5mn.apply(scipy.stats.rankdata))*100

kendall_corr = data_5mn.corr(method='kendall')*100
kendall_p_values = calculate_p_values(data_5mn.apply(scipy.stats.rankdata))*100

# Extracting relevant correlations and p-values for Pearson
pearson_corr_subset = pearson_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
pearson_p_values_subset = pearson_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]

# Combining correlation values and p-values into a single table for Pearson
pearson_table = pearson_corr_subset.round(4).astype(float)
## Values are in percentages

In [44]:
pearson_table

,return,return_abs,return_2,return_2_abs
AD,0.2303,6.1822,0.3288,6.0282
ADX,0.5422,12.5215,0.7565,11.2722
ADXR,0.4775,10.3506,0.6674,9.4668
APO,-1.1190,-8.6646,-1.4385,-8.0345
AROON_UP,0.8419,5.1587,1.1899,4.3838
AROON_DOWN,-0.5498,-2.0141,-0.7353,-2.3266
AROONOSC,-0.8112,-4.1764,-1.1220,-3.9095
BOP,-2.4281,-1.5495,-2.8628,-1.2608
CCI,-0.7299,-4.2814,-0.8987,-3.8805
CMO,-1.3475,-5.8027,-1.4298,-5.3507


In [45]:
# Extracting relevant correlations and p-values for Spearman
spearman_corr_subset = spearman_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
spearman_p_values_subset = spearman_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]

# Combining correlation values and p-values into a single table for Spearman
spearman_table = spearman_corr_subset.round(4).astype(float)
spearman_table

,return,return_abs,return_2,return_2_abs
AD,0.1349,3.8443,0.1360,3.4611
ADX,0.5660,13.8749,0.9197,12.4812
ADXR,0.3951,12.3654,0.7215,11.4323
APO,-1.9226,-1.7436,-3.1956,-1.6693
AROON_UP,3.0540,3.8076,4.1397,3.0356
AROON_DOWN,-3.0236,-1.2474,-4.2651,-1.7959
AROONOSC,-3.6673,-3.0021,-5.1031,-2.8929
BOP,-6.9338,-1.1626,-7.0797,-0.8610
CCI,-6.0143,-3.3158,-7.0040,-3.1302
CMO,-6.1436,-3.7998,-7.2704,-3.7560


In [46]:
data_15mn = df_15mn[columns_of_interest]

# Calculate correlation matrices and their corresponding p-values
pearson_corr = data_15mn.corr(method='pearson')*100
pearson_p_values = calculate_p_values(data_15mn)*100

spearman_corr = data_15mn.corr(method='spearman')*100
spearman_p_values = calculate_p_values(data_15mn.apply(scipy.stats.rankdata))*100

kendall_corr = data_15mn.corr(method='kendall')*100
kendall_p_values = calculate_p_values(data_15mn.apply(scipy.stats.rankdata))*100

# Extracting relevant correlations and p-values for Pearson
pearson_corr_subset = pearson_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
pearson_p_values_subset = pearson_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]


In [47]:
# Combining correlation values and p-values into a single table for Pearson
pearson_table = pearson_corr_subset.round(4).astype(float)
pearson_table
## Values are in percentage

,return,return_abs,return_2,return_2_abs
AD,0.4045,5.3477,0.5756,5.0446
ADX,0.5591,9.0539,0.7549,8.1859
ADXR,0.3205,7.5655,0.4270,7.1218
APO,-1.4886,-7.9227,-2.0320,-7.7182
AROON_UP,1.0623,5.8492,1.6178,4.8052
AROON_DOWN,-0.4415,-1.4719,-0.6423,-1.9467
AROONOSC,-0.8871,-4.3128,-1.3332,-3.9829
BOP,-1.6618,-2.2533,-2.3996,-2.3766
CCI,-0.3451,-4.9823,-0.7989,-4.5756
CMO,-0.7929,-6.4759,-1.1469,-6.1447


In [48]:
# Extracting relevant correlations and p-values for Spearman
spearman_corr_subset = spearman_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
spearman_p_values_subset = spearman_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]

# Combining correlation values and p-values into a single table for Spearman
spearman_table = spearman_corr_subset.round(4).astype(float)
spearman_table

,return,return_abs,return_2,return_2_abs
AD,0.3341,1.4323,0.4888,0.6625
ADX,0.7823,10.1586,1.2179,9.3493
ADXR,0.5256,8.9400,0.7996,8.5685
APO,-1.5941,-1.8884,-2.5756,-2.0905
AROON_UP,3.0202,4.6161,4.1510,3.4892
AROON_DOWN,-2.6812,-0.3157,-3.5996,-0.9906
AROONOSC,-3.5471,-2.9624,-4.8327,-2.6473
BOP,-7.2581,-1.6126,-7.5672,-1.9820
CCI,-6.4392,-3.5874,-7.4339,-3.2522
CMO,-5.7311,-4.6043,-6.6511,-4.6668


In [49]:
df_1hrn['return_2_abs'] = abs(df_1hrn['return_2'])
data_1h = df_1hrn[columns_of_interest]

# Calculate correlation matrices and their corresponding p-values
pearson_corr = data_1h.corr(method='pearson')*100
pearson_p_values = calculate_p_values(data_1h)*100

spearman_corr = data_1h.corr(method='spearman')*100
spearman_p_values = calculate_p_values(data_1h.apply(scipy.stats.rankdata))*100

kendall_corr = data_1h.corr(method='kendall')*100
kendall_p_values = calculate_p_values(data_1h.apply(scipy.stats.rankdata))*100

# Extracting relevant correlations and p-values for Pearson
pearson_corr_subset = pearson_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
pearson_p_values_subset = pearson_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]


In [50]:

# Combining correlation values and p-values into a single table for Pearson
pearson_table = pearson_corr_subset.round(4).astype(float)
pearson_table
## Values are in percentages

,return,return_abs,return_2,return_2_abs
AD,0.8208,4.6261,1.1999,5.5682
ADX,0.0493,13.0779,0.0127,12.2902
ADXR,0.3215,12.3135,0.4616,11.6845
APO,0.7884,-9.2491,1.1266,-8.9153
AROON_UP,1.0358,6.3676,0.6625,5.2066
AROON_DOWN,0.4800,-3.7591,0.8958,-4.2652
AROONOSC,-0.3259,-6.0295,0.1448,-5.6450
BOP,-2.8172,-3.0058,-3.4949,-2.0161
CCI,-0.3787,-5.5981,-0.4354,-5.0279
CMO,0.3033,-7.0675,0.8655,-6.4695


In [51]:
# Extracting relevant correlations and p-values for Spearman
spearman_corr_subset = spearman_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
spearman_p_values_subset = spearman_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]

# Combining correlation values and p-values into a single table for Spearman
spearman_table = spearman_corr_subset.round(4).astype(float)
spearman_table

,return,return_abs,return_2,return_2_abs
AD,1.2813,0.9577,1.5280,1.6014
ADX,0.7435,11.8211,1.0441,10.5264
ADXR,0.3922,11.8966,0.6917,11.1188
APO,0.5397,-3.1807,0.0784,-2.2903
AROON_UP,1.8908,4.7320,2.2030,3.9977
AROON_DOWN,-0.8171,-1.7491,-1.2232,-2.4548
AROONOSC,-1.7589,-3.8279,-2.3083,-3.7888
BOP,-8.7935,-2.7331,-8.0165,-1.9328
CCI,-5.1084,-5.1874,-5.4092,-4.5811
CMO,-3.3224,-5.4298,-3.4371,-4.6843


In [52]:
# Extracting relevant correlations and p-values for Kendall
kendall_corr_subset = kendall_corr.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]
kendall_p_values_subset = kendall_p_values.loc[indicators, ['return', 'return_abs', 'return_2', 'return_2_abs']]

# Combining correlation values and p-values into a single table for Kendall
kendall_table = kendall_corr_subset.round(4).astype(str) + " (p: " + kendall_p_values_subset.astype(str) + ")"
kendall_table

,return,return_abs,return_2,return_2_abs
AD,0.8927 (p: 2.7199999999999998),0.6911 (p: 9.879999999999999),1.0872 (p: 0.84),1.128 (p: 0.58)
ADX,0.4897 (p: 20.0),7.8903 (p: 0.0),0.6915 (p: 7.19),7.0192 (p: 0.0)
ADXR,0.2551 (p: 49.91),7.9377 (p: 0.0),0.4434 (p: 23.31),7.4122 (p: 0.0)
APO,0.374 (p: 35.23),-2.0638 (p: 0.0),-0.03 (p: 89.25),-1.472 (p: 0.01)
AROON_UP,1.304 (p: 0.11),3.2594 (p: 0.0),1.5296 (p: 0.01),2.7638 (p: 0.0)
AROON_DOWN,-0.597 (p: 15.9),-1.1975 (p: 0.26),-0.898 (p: 3.5000000000000004),-1.6905 (p: 0.0)
AROONOSC,-1.2334 (p: 0.24),-2.5705 (p: 0.0),-1.6203 (p: 0.01),-2.5599 (p: 0.0)
BOP,-5.9727 (p: 0.0),-1.8208 (p: 0.0),-5.4705 (p: 0.0),-1.2864 (p: 0.09)
CCI,-3.5221 (p: 0.0),-3.458 (p: 0.0),-3.713 (p: 0.0),-3.0498 (p: 0.0)
CMO,-2.3575 (p: 0.0),-3.5901 (p: 0.0),-2.4437 (p: 0.0),-3.0957 (p: 0.0)


In [53]:
## These indicators are pattern identifiers
#The values in these columns are integers that represent the pattern type:
# +100 for bullish patterns, -100 for bearish patterns, and 0 for no pattern.
df_5mn_c=df_5m.copy()

df_5mn_c['CDL2CROWS'] = talib.CDL2CROWS(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDL3BLACKCROWS'] = talib.CDL3BLACKCROWS(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDL3INSIDE'] = talib.CDL3INSIDE(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDL3LINESTRIKE'] = talib.CDL3LINESTRIKE(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDL3OUTSIDE'] = talib.CDL3OUTSIDE(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDL3STARSINSOUTH'] = talib.CDL3STARSINSOUTH(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDL3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDLABANDONEDBABY'] = talib.CDLABANDONEDBABY(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDLADVANCEBLOCK'] = talib.CDLADVANCEBLOCK(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
df_5mn_c['CDLBELTHOLD'] = talib.CDLBELTHOLD(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])
# ... Continue for the rest of the patterns
# Breakaway
df_5mn_c['CDLBREAKAWAY'] = talib.CDLBREAKAWAY(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])        
#  Closing Marubozu
df_5mn_c['CDLCLOSINGMARUBOZU']= talib.CDLCLOSINGMARUBOZU(df_5mn_c['open'], df_5mn_c['high'], df_5mn_c['low'], df_5mn_c['close'])        
# df_5mn[CDLCONCEALBABYSWALL]  Concealing Baby Swallow
# df_5mn[CDLCOUNTERATTACK]     Counterattack
# df_5mn[CDLDARKCLOUDCOVER]    Dark Cloud Cover
# df_5mn[CDLDOJI]              Doji
# df_5mn[CDLDOJISTAR]          Doji Star
# df_5mn[CDLDRAGONFLYDOJI]     Dragonfly Doji
# df_5mn[CDLENGULFING]         Engulfing Pattern
# df_5mn[CDLEVENINGDOJISTAR]   Evening Doji Star
# df_5mn[CDLEVENINGSTAR]       Evening Star
# df_5mn[CDLGAPSIDESIDEWHITE]  Up/Down-gap side-by-side white lines
# df_5mn[CDLGRAVESTONEDOJI]    Gravestone Doji
# df_5mn[CDLHAMMER]            Hammer
# df_5mn[CDLHANGINGMAN]        Hanging Man
# df_5mn[CDLHARAMI]            Harami Pattern
# df_5mn[CDLHARAMICROSS]       Harami Cross Pattern
# df_5mn[CDLHIGHWAVE]          High-Wave Candle
# df_5mn[CDLHIKKAKE]           Hikkake Pattern
# df_5mn[CDLHIKKAKEMOD]        Modified Hikkake Pattern
# df_5mn[CDLHOMINGPIGEON]      Homing Pigeon
# df_5mn[CDLIDENTICAL3CROWS]   Identical Three Crows
# df_5mn[CDLINNECK]            In-Neck Pattern
# df_5mn[CDLINVERTEDHAMMER]    Inverted Hammer
# df_5mn[CDLKICKING]           Kicking
# df_5mn[CDLKICKINGBYLENGTH]   Kicking - bull/bear determined by the longer marubozu
# df_5mn[CDLLADDERBOTTOM]      Ladder Bottom
# df_5mn[CDLLONGLEGGEDDOJI]    Long Legged Doji
# df_5mn[CDLLONGLINE]          Long Line Candle
# df_5mn[CDLMARUBOZU]         Marubozu
# df_5mn[CDLMATCHINGLOW]       Matching Low
# df_5mn[CDLMATHOLD]           Mat Hold
# df_5mn[CDLMORNINGDOJISTAR]   Morning Doji Star
# df_5mn[CDLMORNINGSTAR]       Morning Star
# df_5mn[CDLONNECK]            On-Neck Pattern
# df_5mn[CDLPIERCING]          Piercing Pattern
# df_5mn[CDLRICKSHAWMAN]       Rickshaw Man
# df_5mn[CDLRISEFALL3METHODS]  Rising/Falling Three Methods
# df_5mn[CDLSEPARATINGLINES]   Separating Lines
# df_5mn[CDLSHOOTINGSTAR]      Shooting Star
# df_5mn[CDLSHORTLINE]         Short Line Candle
# df_5mn[CDLSPINNINGTOP]       Spinning Top
# df_5mn[CDLSTALLEDPATTERN]    Stalled Pattern
# df_5mn[CDLSTICKSANDWICH]     Stick Sandwich
# df_5mn[CDLTAKURI]            Takuri (Dragonfly Doji with very long lower shadow)
# df_5mn[CDLTASUKIGAP]         Tasuki Gap
# df_5mn[CDLTHRUSTING]         Thrusting Pattern
# df_5mn[CDLTRISTAR]           Tristar Pattern
# df_5mn[CDLUNIQUE3RIVER]      Unique 3 River
# df_5mn[CDLUPSIDEGAP2CROWS]   Upside Gap Two Crows
# df_5mn[CDLXSIDEGAP3METHODS]  Upside/Downside Gap Three Methods



In [54]:
df_5mn

,open,high,low,close,volume,return,return_abs,return_2,return_2_abs,AD,...,RSI,STOCH_K,STOCH_D,STOCHF_K,STOCHF_D,STOCHRSI_K,STOCHRSI_D,TRIX,ULTOSC,WILLR
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 12:50:00,13681.62,13700.00,13605.09,13669.36,18.434229,0.002095,0.002095,0.005460,0.005460,-9.085314e+01,...,51.949498,46.588620,63.548420,31.736445,46.588620,0.000000,-4.736952e-15,0.035889,61.652967,-48.432022
2018-01-01 12:55:00,13674.92,13775.99,13631.97,13698.00,23.361626,0.003358,0.003358,-0.002743,0.002743,-9.279318e+01,...,54.740295,37.974529,50.009219,44.840997,37.974529,16.152884,5.384295e+00,0.036460,58.422678,-45.244708
2018-01-01 13:00:00,13698.00,13775.00,13690.00,13744.00,29.418698,-0.006080,0.006080,-0.007207,0.007207,-8.483283e+01,...,58.872070,47.591736,44.051629,66.197766,47.591736,54.022269,2.339172e+01,0.037003,57.913445,-34.111187
2018-01-01 13:05:00,13721.28,13744.00,13650.00,13660.43,31.432075,-0.001133,0.001133,-0.005887,0.005887,-1.092897e+02,...,49.950487,48.458647,44.674971,34.337178,48.458647,0.000000,2.339172e+01,0.037339,53.592066,-72.349577
2018-01-01 13:10:00,13678.87,13679.00,13590.00,13644.95,28.925881,-0.004759,0.004759,-0.002489,0.002489,-1.024970e+02,...,48.484793,43.359848,46.470077,29.544599,43.359848,0.000000,1.800742e+01,0.037461,53.685177,-75.957121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 23:30:00,36935.81,36955.02,36805.73,36890.24,190.291204,-0.001117,0.001117,-0.003539,0.003539,1.748522e+06,...,48.496406,86.029048,78.688390,73.846340,86.029048,70.695600,9.023187e+01,0.015237,58.482111,-57.042100
2021-05-31 23:35:00,36890.85,36893.32,36800.40,36849.03,144.079780,-0.002425,0.002425,-0.001278,0.001278,1.748529e+06,...,46.508659,74.946582,81.422551,57.208608,74.946582,14.384744,6.169345e+01,0.014423,62.681714,-65.206538
2021-05-31 23:40:00,36849.04,36878.94,36750.46,36759.67,221.044999,0.001149,0.001149,0.000393,0.000393,1.748340e+06,...,42.445972,45.185765,68.720465,4.502346,45.185765,0.000000,2.836011e+01,0.013539,58.585015,-77.514793


In [55]:
df_15mn['rel_vol_change']= df_15mn['volume'].shift(-1) / df_15mn['volume']-1
df_15mn['rel_vol_change_2']= df_15mn['volume'].shift(-2) / df_15mn['volume']-1


pearson_correlation_rel_vol_change = df_15mn[['rel_vol_change', 'rel_vol_change_2','return', 'return_2', 'return_abs', 'return_2_abs']].corr().loc['rel_vol_change']
spearman_correlation_rel_vol_change = df_15mn[['rel_vol_change','rel_vol_change_2', 'return', 'return_2', 'return_abs', 'return_2_abs']].corr(method='spearman').loc['rel_vol_change']

pearson_correlation_rel_vol_change*100, spearman_correlation_rel_vol_change*100


(rel_vol_change      100.000000
 rel_vol_change_2     53.629473
 return               -3.916829
 return_2             -3.364943
 return_abs           27.085775
 return_2_abs         17.930696
 Name: rel_vol_change, dtype: float64,
 rel_vol_change      100.000000
 rel_vol_change_2     57.509378
 return               -3.115164
 return_2             -2.327832
 return_abs           21.519317
 return_2_abs         11.270531
 Name: rel_vol_change, dtype: float64)

In [56]:
pearson_correlation_rel_vol_change = df_15mn[['rel_vol_change', 'rel_vol_change_2','return', 'return_2', 'return_abs', 'return_2_abs']].corr().loc['rel_vol_change_2']
spearman_correlation_rel_vol_change = df_15mn[['rel_vol_change','rel_vol_change_2', 'return', 'return_2', 'return_abs', 'return_2_abs']].corr(method='spearman').loc['rel_vol_change_2']

pearson_correlation_rel_vol_change*100, spearman_correlation_rel_vol_change*100


(rel_vol_change       53.629473
 rel_vol_change_2    100.000000
 return               -3.277088
 return_2             -5.949368
 return_abs           14.372015
 return_2_abs         32.331683
 Name: rel_vol_change_2, dtype: float64,
 rel_vol_change       57.509378
 rel_vol_change_2    100.000000
 return               -2.127337
 return_2             -3.478007
 return_abs            9.182945
 return_2_abs         24.614793
 Name: rel_vol_change_2, dtype: float64)

## Alpha Based Indicators

Determining Indicators based on custom formulations which have worked well on US Stock market and identifying

In [57]:
def calculate_ic_and_correlations_df(df, frequency='15T'):
    """
    Calculates the Information Coefficient (IC) and correlation coefficients (Pearson, Spearman, Kendall)
    between alpha values and next day returns using a single dataframe that includes both OHLCV data and alpha values.

    :param df: DataFrame containing the OHLCV data and alpha values
    :param frequency: The frequency of the data ('5T' for 5 minutes, '15T' for 15 minutes, etc.)
    :return: Prints the IC and correlation coefficients with 3 decimal points and as percentages
    """
    # Calculate the number of periods in one day
    periods_per_day = pd.Timedelta('1D') // pd.Timedelta(frequency)

    # Calculate next day returns
    df['next_day_return'] = df['close'].shift(-periods_per_day) / df['close'] - 1

    # Drop the rows with NaN values that occur due to shifting
    df.dropna(subset=['next_day_return', 'alpha'], inplace=True)

    # Calculate correlations
    pearson_corr = df['alpha'].corr(df['next_day_return'], method='pearson')
    spearman_corr = df['alpha'].corr(df['next_day_return'], method='spearman')
    kendall_corr = df['alpha'].corr(df['next_day_return'], method='kendall')

    # Calculate Information Coefficient (IC) as the mean of daily Pearson correlations
    # Group by date to consider the high frequency data as daily data
    daily_corrs = df.groupby(df.index.date).apply(
        lambda x: x['alpha'].corr(x['next_day_return'], method='pearson'))
    ic = daily_corrs.mean()

    # Output the results with 3 decimal points and as percentages
    print(f'Information Coefficient (IC): {ic:.3f}')
    print(f'Pearson Correlation (%): {pearson_corr*100:.3f}%')
    print(f'Spearman Correlation (%): {spearman_corr*100:.3f}%')
    print(f'Kendall Correlation (%): {kendall_corr*100:.3f}%')




In [ ]:
df_alp1=df_15m.copy()
df_alp1['alpha'] = np.maximum(
(df_alp1['low'] / df_alp1['open']) * (df_alp1['high'] / df_alp1['close']),
        np.cos(np.minimum(df_alp1['close'], df_alp1['open']))
    )
calculate_ic_and_correlations_df(df_alp1)

Information Coefficient (IC): -0.068
Pearson Correlation (%): 0.931%
Spearman Correlation (%): 0.111%
Kendall Correlation (%): 0.069%


In [ ]:
df_alp101=df_15m.copy()
df_alp101['alpha'] = (df_alp101['close']-df_alp101['open'])/(df_alp101['high']-df_alp101['low'])
calculate_ic_and_correlations_df(df_alp101)

Information Coefficient (IC): -0.090
Pearson Correlation (%): 0.197%
Spearman Correlation (%): -0.488%
Kendall Correlation (%): -0.328%


In [ ]:
def calculate_vwap(df):
    vwap = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum() / df['volume'].cumsum()
    return vwap

# Function to calculate Alpha#41
def calculate_alpha_41(df):

    df['VWAP'] = calculate_vwap(df)
    df['alpha'] = np.sqrt(df['high'] * df['low'] - df['VWAP'])
    return df

df_alp41=df_15m.copy()
df_alp41=calculate_alpha_41(df_alp41)
calculate_ic_and_correlations_df(df_alp41)

Information Coefficient (IC): -0.652
Pearson Correlation (%): -0.723%
Spearman Correlation (%): -0.447%
Kendall Correlation (%): -0.369%


In [ ]:
df_alp2=df_15m.copy()
df_alp2['Previous Close'] = df_alp2['close'].shift(1)
df_alp2['Return'] = (df_alp2['close'] - df_alp2['Previous Close']) / df_alp2['Previous Close']
df_alp2['alpha'] = ((df_alp2['low'] / df_alp2['close']) - df_alp2['Return']) +((df_alp2['low'] / df_alp2['close']) * np.cos(df_alp2['Return']))
calculate_ic_and_correlations_df(df_alp2)


Information Coefficient (IC): 0.064
Pearson Correlation (%): -1.232%
Spearman Correlation (%): -0.189%
Kendall Correlation (%): -0.084%


In [ ]:
df_alp3=df_15m.copy()
df_alp3['alpha'] = np.minimum(0.938,df_alp3['low'] / df_alp3['close'])
calculate_ic_and_correlations_df(df_alp3)

Information Coefficient (IC): -0.001
Pearson Correlation (%): -1.308%
Spearman Correlation (%): -0.715%
Kendall Correlation (%): -0.584%


In [ ]:
df_alp4=df_15m.copy()
df_alp4['alpha'] = np.minimum(df_alp4['low'] - df_alp4['open'], 0.133) + np.maximum(np.cos(df_alp4['volume']), df_alp4['high'] - df_alp4['close'])
calculate_ic_and_correlations_df(df_alp4)


Information Coefficient (IC): -0.067
Pearson Correlation (%): 0.800%
Spearman Correlation (%): 0.085%
Kendall Correlation (%): 0.043%


In [ ]:
df_alp5=df_15m.copy()
df_alp5['alpha'] = np.maximum(np.minimum(df_alp5['low'] - df_alp5['open'], 0.133),(df_alp5['low'] - df_alp5['close']) / np.sqrt(df_alp5['high']))
calculate_ic_and_correlations_df(df_alp5)

Information Coefficient (IC): 0.006
Pearson Correlation (%): -1.476%
Spearman Correlation (%): -1.031%
Kendall Correlation (%): -0.654%


In [ ]:
df_alp6=df_15m.copy()
df_alp6['Previous Close'] = df_alp6['close'].shift(1)
df_alp6['Return'] = (df_alp6['close'] - df_alp6['Previous Close']) / df_alp6['Previous Close']
df_alp6['alpha'] = ((df_alp6['high'] - df_alp6['close']) - df_alp6['Return'])/((df_alp6['low'] + df_alp6['Return']) + (-0.177 / df_alp6['close']))
calculate_ic_and_correlations_df(df_alp6)

Information Coefficient (IC): 0.082
Pearson Correlation (%): 4.081%
Spearman Correlation (%): 1.656%
Kendall Correlation (%): 1.107%


In [ ]:
df_alp7=df_15m.copy()
df_alp7['Previous Close'] = df_alp7['close'].shift(1)
df_alp7['Return'] = (df_alp7['close'] - df_alp7['Previous Close']) / df_alp7['Previous Close']

df_alp7['alpha'] =np.maximum((df_alp7['high'] / df_alp7['close']) - df_alp7['Return'].abs(),
                             np.minimum(df_alp7['low'] / df_alp7['open'], df_alp7['high'] / df_alp7['close']))
calculate_ic_and_correlations_df(df_alp7)

Information Coefficient (IC): 0.025
Pearson Correlation (%): 2.606%
Spearman Correlation (%): 0.670%
Kendall Correlation (%): 0.467%


In [ ]:
df_alp8=df_15m.copy()
df_alp8['alpha'] =0.386/df_alp8['low'] + df_alp8['low']/df_alp8['close']
calculate_ic_and_correlations_df(df_alp8)

Information Coefficient (IC): 0.016
Pearson Correlation (%): -2.231%
Spearman Correlation (%): -0.980%
Kendall Correlation (%): -0.626%


In [ ]:
df_alp54=df_15m.copy()
df_alp54['alpha'] =(-1)*(df_alp54['low']-df_alp54['close'])*((df_alp54['open']/df_alp54['close'])**5)/(df_alp54['low']-df_alp54['high'])
calculate_ic_and_correlations_df(df_alp54)

Information Coefficient (IC): 0.055
Pearson Correlation (%): -0.031%
Spearman Correlation (%): 0.511%
Kendall Correlation (%): 0.340%


In [ ]:
df_alp9=df_15m.copy()
df_alp9['alpha'] = (df_alp9['close']-df_alp9['open'])/(df_alp9['high']-df_alp9['low'])+0.001
calculate_ic_and_correlations_df(df_alp9)

Information Coefficient (IC): -0.090
Pearson Correlation (%): 0.197%
Spearman Correlation (%): -0.488%
Kendall Correlation (%): -0.328%


In [ ]:
def delta(series, period):
    return series.diff(period)

# Define sign function
def sign(series):
    return series.apply(np.sign)

# Define correlation function
def correlation(series1, series2, period):
    return series1.rolling(window=period).corr(series2)


# Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1)))
df_alp12=df_15m.copy()
df_alp12['alpha'] = sign(delta(df_alp12['volume'], 1)) * (-1 * delta(df_15m['close'], 1))
calculate_ic_and_correlations_df(df_alp12)

Information Coefficient (IC): 0.100
Pearson Correlation (%): -0.007%
Spearman Correlation (%): 0.371%
Kendall Correlation (%): 0.280%


In [ ]:
# Alpha#13/6: (-1 * correlation(open, volume, 10))
df_alp13=df_15m.copy()
df_alp13['alpha'] = -1 * correlation(df_alp13['open'], df_alp13['volume'], 10)
calculate_ic_and_correlations_df(df_alp13)


Information Coefficient (IC): 0.174
Pearson Correlation (%): -1.703%
Spearman Correlation (%): 0.707%
Kendall Correlation (%): 0.485%
